# Data Preprocessing

In [ ]:
import pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import chi2_contingency
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
df=pd.read_csv(r'/Users/mrkashif/Documents/Zahra_docs/MINS/MINS_Project/postoperative_final_tbu_dataset.csv')
df_copy = df.copy()

In [ ]:
df['MINS_final'].value_counts().plot(kind='bar')
plt.title('label balance')
plt.xlabel('label values')
plt.ylabel('amount per label')
plt.show()
print(df['MINS_final'].value_counts())

In [ ]:
# If any yes/columns are present then change it to numeric type.
def convert_yn_to_01(row, column_name):
    return 1 if row[column_name] == 'Female' else 0

df_new1 = df_copy.copy()

In [ ]:
bool_columns = ['gender_desc_at_surg_checkin']
for bool_col in bool_columns:
    df_new1[bool_col + '_updated'] = df.apply(lambda row: convert_yn_to_01(row, bool_col),axis=1)

In [ ]:
df_new1 = df_new1.drop(columns = bool_columns)
df_new1 = df_new1.rename(columns={'gender_desc_at_surg_checkin_updated': 'gender_updated'})

In [ ]:
df_new1

In [ ]:
df_new1['standardized_collection_dt_tm']= pd.to_datetime(df_new1['standardized_collection_dt_tm'])
df_new1['disch_time_surg_end']= pd.to_datetime(df_new1['disch_time_surg_end'])
df_new1['standardized_clinical_time_lead']= pd.to_datetime(df_new1['standardized_clinical_time_lead'])
df_new1['standardized_clinical_event_time']= pd.to_datetime(df_new1['standardized_clinical_event_time'])
df_new1['clinical_event_performed_dt_tm']= pd.to_datetime(df_new1['clinical_event_performed_dt_tm'])
df_new1['standardized_surg_stop_time']= pd.to_datetime(df_new1['standardized_surg_stop_time'])
df_new1['surg_stop_dt_tm']= pd.to_datetime(df_new1['surg_stop_dt_tm'])
df_new1['surg_start_dt_tm']= pd.to_datetime(df_new1['surg_start_dt_tm'])
df_new1['patient_out_room_dt_tm']= pd.to_datetime(df_new1['patient_out_room_dt_tm'])
df_new1['patient_in_room_dt_tm']= pd.to_datetime(df_new1['patient_in_room_dt_tm'])
df_new1['disch_dt_tm']= pd.to_datetime(df_new1['disch_dt_tm'])
df_new1['admit_dt_tm']= pd.to_datetime(df_new1['admit_dt_tm'])
df_new1['surgery_performed_date']= pd.to_datetime(df_new1['surgery_performed_date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

category_data = df_new1.select_dtypes(include='object').columns
numerical_data= df_new1.select_dtypes(exclude='object').columns

df_new2 = df_new1[category_data].apply(LabelEncoder().fit_transform)
df_new2

In [ ]:
final_data_set = pd.DataFrame()
for col in category_data:
    if col != 'lag_st_time' and col != 'lead_st_time':
        print(col, len(df_new2[col].unique()))
        col_vals = [vals for vals in df_new2[col].unique()]
        onehot_cols = [col + '_' + str(s) for s in col_vals]
        dummies_train = pd.get_dummies(df_new2[col], prefix=col)
        train_data = pd.get_dummies(df_new2[col], prefix=col)
        final_data_set = pd.concat([final_data_set , dummies_train], axis=1)

In [ ]:
final_data = pd.concat([final_data_set,df_new1[numerical_data]], axis = 1)
final_data

In [ ]:
import datetime as dt
final_data['standardized_collection_dt_tm'] = pd.to_datetime(final_data['standardized_collection_dt_tm'])
final_data['standardized_collection_dt_tm']=final_data['standardized_collection_dt_tm'].map(dt.datetime.toordinal)

final_data['disch_time_surg_end'] = pd.to_datetime(final_data['disch_time_surg_end'])
final_data['disch_time_surg_end']=final_data['disch_time_surg_end'].map(dt.datetime.toordinal)

final_data['standardized_clinical_time_lead'] = pd.to_datetime(final_data['standardized_clinical_time_lead'])
final_data['standardized_clinical_time_lead']=final_data['standardized_clinical_time_lead'].map(dt.datetime.toordinal)

final_data['standardized_clinical_event_time'] = pd.to_datetime(final_data['standardized_clinical_event_time'])
final_data['standardized_clinical_event_time']=final_data['standardized_clinical_event_time'].map(dt.datetime.toordinal)

final_data['clinical_event_performed_dt_tm'] = pd.to_datetime(final_data['clinical_event_performed_dt_tm'])
final_data['clinical_event_performed_dt_tm']=final_data['clinical_event_performed_dt_tm'].map(dt.datetime.toordinal)

final_data['standardized_surg_stop_time'] = pd.to_datetime(final_data['standardized_surg_stop_time'])
final_data['standardized_surg_stop_time']=final_data['standardized_surg_stop_time'].map(dt.datetime.toordinal)

final_data['surg_stop_dt_tm'] = pd.to_datetime(final_data['surg_stop_dt_tm'])
final_data['surg_stop_dt_tm']=final_data['surg_stop_dt_tm'].map(dt.datetime.toordinal)

final_data['surg_start_dt_tm'] = pd.to_datetime(final_data['surg_start_dt_tm'])
final_data['surg_start_dt_tm']=final_data['surg_start_dt_tm'].map(dt.datetime.toordinal)

final_data['patient_out_room_dt_tm'] = pd.to_datetime(final_data['patient_out_room_dt_tm'])
final_data['patient_out_room_dt_tm']=final_data['patient_out_room_dt_tm'].map(dt.datetime.toordinal)

final_data['patient_in_room_dt_tm'] = pd.to_datetime(final_data['patient_in_room_dt_tm'])
final_data['patient_in_room_dt_tm']=final_data['patient_in_room_dt_tm'].map(dt.datetime.toordinal)

final_data['disch_dt_tm'] = pd.to_datetime(final_data['disch_dt_tm'])
final_data['disch_dt_tm']=final_data['disch_dt_tm'].map(dt.datetime.toordinal)

final_data['admit_dt_tm'] = pd.to_datetime(final_data['admit_dt_tm'])
final_data['admit_dt_tm']=final_data['admit_dt_tm'].map(dt.datetime.toordinal)

final_data['surgery_performed_date'] = pd.to_datetime(final_data['surgery_performed_date'])
final_data['surgery_performed_date']=final_data['surgery_performed_date'].map(dt.datetime.toordinal)

final_data['MAP_Non_Invasive_calc'] = pd.to_datetime(final_data['MAP_Non_Invasive_calc'])
final_data['MAP_Non_Invasive_calc']=final_data['MAP_Non_Invasive_calc'].map(dt.datetime.toordinal)



In [ ]:
final_data

In [ ]:
#feature selection for vitals
data_to_use = final_data[['age_at_surg_checkin',
                          'peripheral_pulse_rate', 
                          'Respiratory_Rate_br_per_min',
                          'SpO2', 
                          'map_rec_final',
                          'sbp_final', 
                          'dbp_final', 
                          'map_cal_final', 
                          'sbp_final_min', 
                          'dbp_final_min', 
                          'map_final_min',
                          'pulse_rate_max', 
                          'gender_updated',
                          'anesthesia_type_desc_0',
                           'anesthesia_type_desc_1',
                            'anesthesia_type_desc_2',
                            'anesthesia_type_desc_3',
                            'anesthesia_type_desc_4',
                            'anesthesia_type_desc_5',
                            'anesthesia_type_desc_6',
                            'anesthesia_type_desc_7',
                            'anesthesia_type_desc_8',
                            'anesthesia_type_desc_9',
                            'anesthesia_type_desc_10',
                            'sched_surg_priority_desc_0',
                            'sched_surg_priority_desc_1',
                            'sched_surg_priority_desc_2',
                            'sched_surg_priority_desc_3',
                            'sched_surg_priority_desc_4',
                            'sched_surg_priority_desc_5',
                            'disch_disp_desc_src_0',
                            'disch_disp_desc_src_1',
                            'disch_disp_desc_src_2',
                            'disch_disp_desc_src_3',
                            'disch_disp_desc_src_4',
                            'disch_disp_desc_src_5',
                            'disch_disp_desc_src_6',
                            'disch_disp_desc_src_7',
                            'disch_disp_desc_src_8',
                            'disch_disp_desc_src_9',
                            'disch_disp_desc_src_10',
                            'disch_disp_desc_src_11',
                            'is_surg_case_major',
                            'surg_start_to_stop_elapsed_time_minutes',
                            'diff_in_seconds',
                            'max_pulse_variation'
                          
                          
                         ]]
data_to_use

In [ ]:
# remove rows with null/nan values
data_to_use = data_to_use.dropna()
data_to_use

In [ ]:
data_to_use.columns

In [ ]:
#!/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install xgboost
#!/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install xgboost==0.90
# !/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
# !/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
# !/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
# !/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64


# ML Models

In [ ]:
#Linear Regression
linear_model = LinearRegression()

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
random_forest_model = RandomForestRegressor(n_estimators=20, random_state=0)

In [ ]:
#Boosting
from xgboost import XGBRegressor
xgboost_model = XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
# the most commonly configured hyperparameters are the following:

# n_estimators: The number of trees in the ensemble, often increased until no further improvements are seen.
# max_depth: The maximum depth of each tree, often values are between 1 and 10.

# eta: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.

# subsample: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to 
# use all samples.

# colsample_bytree: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 
# to use all features.


In [ ]:
#Bagging
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(n_estimators=100, random_state=12)

# Introduce missing values in the dataset

In [ ]:
def dbp_missing(final_data, x,y, frac):
    
    #store datasets with missing values(dbp)
    y_data = final_data.iloc[:, 6].values
    X_data = final_data.iloc[:, np.r_[0:6,7:46]].values
    
    df_copy = final_data.copy()
    df_copy[x] = df_copy[x].sample(frac=frac, random_state=12)
    df_copy[x].isna().sum()
    df_copy.rename(columns={x: y}, inplace=True)
    df_copy[y]
    df_dbp = pd.concat([final_data, df_copy[y]], axis=1)
    actual = df_dbp[x]
    pred = df_dbp[y]
    return df_dbp
    

In [ ]:
# generating datasets with 5,10,20,30,40,50% of missing values for dbp , originally idx 6
dbp_5 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.95)
dbp_10 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.90)
dbp_20 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.80)
dbp_30 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.70)
dbp_40 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.60)
dbp_50 = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.50)

In [ ]:
dbp_50

In [ ]:
dbp_x = dbp_missing(data_to_use, 'dbp_final', 'dbp_ampute', 0.50)
dbp_x

In [ ]:
def sbp_missing(final_data, x,y, frac):
    
    #store datasets with missing values(dbp)
    y_data = final_data.iloc[:, 5].values
    X_data = final_data.iloc[:, np.r_[0:5,6:46]].values
    
    df_copy = final_data.copy()
    df_copy[x] = df_copy[x].sample(frac=frac, random_state=12)
    df_copy[x].isna().sum()
    df_copy.rename(columns={x: y}, inplace=True)
    df_copy[y]
    df_sbp = pd.concat([final_data, df_copy[y]], axis=1)
    actual = df_sbp[x]
    pred = df_sbp[y]
    return df_sbp
    

In [ ]:
# generating datasets with 5,10,20,30,40,50% of missing values for sbp , originally idx 5
sbp_5 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.95)
sbp_10 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.90)
sbp_20 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.80)
sbp_30 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.70)
sbp_40 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.60)
sbp_50 = sbp_missing(data_to_use, 'sbp_final', 'sbp_ampute', 0.50)

In [ ]:
def pulse_missing(final_data, x,y, frac):
    
    #store datasets with missing values(dbp)
    y_data = final_data.iloc[:, 1].values
    X_data = final_data.iloc[:, np.r_[0:1,2:46]].values
    
    df_copy = final_data.copy()
    df_copy[x] = df_copy[x].sample(frac=frac, random_state=12)
    df_copy[x].isna().sum()
    df_copy.rename(columns={x: y}, inplace=True)
    df_copy[y]
    df_sbp = pd.concat([final_data, df_copy[y]], axis=1)
    actual = df_sbp[x]
    pred = df_sbp[y]
    return df_sbp
    

In [ ]:
# generating datasets with 5,10,20,30,40,50% of missing values for pulse , originally idx 5
pulse_5 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.95)
pulse_10 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.90)
pulse_20 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.80)
pulse_30 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.70)
pulse_40 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.60)
pulse_50 = pulse_missing(data_to_use, 'peripheral_pulse_rate', 'pulse_ampute', 0.50)

In [ ]:
def resp_missing(final_data, x,y, frac):
    
    #store datasets with missing values(dbp)
    y_data = final_data.iloc[:, 2].values
    X_data = final_data.iloc[:, np.r_[0:2,3:46]].values
    
    df_copy = final_data.copy()
    df_copy[x] = df_copy[x].sample(frac=frac, random_state=12)
    df_copy[x].isna().sum()
    df_copy.rename(columns={x: y}, inplace=True)
    df_copy[y]
    df_sbp = pd.concat([final_data, df_copy[y]], axis=1)
    actual = df_sbp[x]
    pred = df_sbp[y]
    return df_sbp
    

In [ ]:
# generating datasets with 5,10,20,30,40,50% of missing values for resp , originally idx 5
resp_5 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.95)
resp_10 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.90)
resp_20 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.80)
resp_30 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.70)
resp_40 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.60)
resp_50 = resp_missing(data_to_use, 'Respiratory_Rate_br_per_min', 'resp_ampute', 0.50)

In [ ]:
data_to_use

In [ ]:
folds_5 = {
    'fold_1': {},
    'fold_2': {},
    'fold_3': {},
    'fold_4': {},
    'fold_5': {} 
}

folds_10 = {
    'fold_1': {},
    'fold_2': {},
    'fold_3': {},
    'fold_4': {},
    'fold_5': {},
    'fold_6': {},
    'fold_7': {},
    'fold_8': {},
    'fold_9': {},
    'fold_10': {}
}

In [ ]:
k = 3
y_data = final_data.iloc[:, 46].values
X_data = final_data.iloc[:, np.r_[0:46]].values
k_folds = KFold(n_splits = k, shuffle=True, random_state=0)
print(k_folds.split(X_data, y_data))

# Divide into k-folds

In [ ]:
# shuffle shuffle=True the data and divide into k folds (5) n_splits=5
# the k folds should have the same partitions, so we set random_state=0
from sklearn.model_selection import KFold
def k_folds_cross(final_data, k, folds):
    y_data = final_data.iloc[:, 46].values
    X_data = final_data.iloc[:, np.r_[0:46]].values
    k_folds = KFold(n_splits = k, shuffle=True, random_state=0)
    
    i = 1
    for train_index, test_index in k_folds.split(X_data, y_data):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]
        fold_num = 'fold_'+str(i)
        folds[fold_num] = {
           'X_train': X_train,
           'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test   
        }   

        i = i+1
    return folds

In [ ]:
columns = ['age_at_surg_checkin',
                          'peripheral_pulse_rate', 
                          'Respiratory_Rate_br_per_min',
                          'SpO2', 
                          'map_rec_final',
                          'sbp_final', 
                          'dbp_final', 
                          'map_cal_final', 
                          'sbp_final_min', 
                          'dbp_final_min', 
                          'map_final_min',
                          'pulse_rate_max', 
                          'gender_updated',
                          'anesthesia_type_desc_0',
                           'anesthesia_type_desc_1',
                            'anesthesia_type_desc_2',
                            'anesthesia_type_desc_3',
                            'anesthesia_type_desc_4',
                            'anesthesia_type_desc_5',
                            'anesthesia_type_desc_6',
                            'anesthesia_type_desc_7',
                            'anesthesia_type_desc_8',
                            'anesthesia_type_desc_9',
                            'anesthesia_type_desc_10',
                            'sched_surg_priority_desc_0',
                            'sched_surg_priority_desc_1',
                            'sched_surg_priority_desc_2',
                            'sched_surg_priority_desc_3',
                            'sched_surg_priority_desc_4',
                            'sched_surg_priority_desc_5',
                            'disch_disp_desc_src_0',
                            'disch_disp_desc_src_1',
                            'disch_disp_desc_src_2',
                            'disch_disp_desc_src_3',
                            'disch_disp_desc_src_4',
                            'disch_disp_desc_src_5',
                            'disch_disp_desc_src_6',
                            'disch_disp_desc_src_7',
                            'disch_disp_desc_src_8',
                            'disch_disp_desc_src_9',
                            'disch_disp_desc_src_10',
                            'disch_disp_desc_src_11',
                            'is_surg_case_major',
                            'surg_start_to_stop_elapsed_time_minutes',
                            'diff_in_seconds',
                            'max_pulse_variation'
                          
                          
                         ]

# Bootstrap validation 

In [ ]:
# we did a variation of the bootstrapsampling
# bv = split data to train and test, sample on train, 100 iters
# bv_o = original method of bootstrap, sample data, pick out unused data and make that your test
# 5_fold_bv = we first do 5 folds, then we resample the train set, and have 10 iterations  in each fold

In [ ]:
# Example of the resampling

data = pd.DataFrame({'num_legs': [1,2,3,4,5,6,7,8,9],
                   'num_wings':[1,None,3,4,None,6,7,8,9],
                   'num_specimen_seen': [1,2,3,4,5,6,7,8,9]},
                  )
y = data['num_wings']
data.drop('num_wings', inplace=True, axis=1)
# y = data['dbp_ampute'] # example dbp_ampute, this is the target with null values
# data.drop('dbp_ampute', inplace=True, axis=1)
X = data
n_samples = X.shape[0]
n_bootstraps = 3
import time
start = time.perf_counter()
for i in range(n_bootstraps):
    # Sample the data with replacement
    X_resampled, y_resampled = resample(X, y, replace=True)

    # Use the leftover observations as the test set
    X_test = X[~data.index.isin(X_resampled.index)]
    y_test = y[~data.index.isin(y_resampled.index)]
#     X_test = X[~np.isin(X, X_resampled)]
#     y_test = y[~np.isin(y, y_resampled)]
#     print(y_test)
#     print("-------")
#     print(y_resampled)
end = time.perf_counter()
elapsed = end - start

# Print the elapsed time
print(f"Elapsed time: {elapsed:.3f} seconds")



In [ ]:
# Sample of the resampling method
from sklearn.utils import resample
data = [1,2,3,4,5,6,7] # ,8,9,10]
n_iterations = 10       
n_size = int(len(data) * 1)
stats = list()
for i in range(n_iterations):
    train = resample(data, n_samples=n_size)#, random_state=0)   
    test = np.array([x for x in data if x not in train]) 
    print("Train_data ->", train, " " , "Test_data ->", test)

In [ ]:
# Fixed test set BV sample
from sklearn.utils import resample
df = pd.DataFrame({'num_legs': [1,2,3,4,5,6,7,8,9],
                   'num_wings':[1,2,3,4,5,6,7,8,9],
                   'num_specimen_seen': [1,2,3,4,5,6,7,8,9]},
                  )

X = df['num_legs']
y = df['num_specimen_seen']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)
print(X_train.values)
print(y_train.values)
new_x, new_y = resample(X_train, y_train, n_samples=9)
print(new_x.values)
print(new_y.values)

In [ ]:
#sample
from sklearn.model_selection import KFold

from sklearn.utils import resample
iters = {}
# Create an array of data and corresponding target values
data = pd.DataFrame({'num_legs': [0,1,2,3,4,5,6,7,8,9],
                   'num_wings':[0,1,2,3,4,5,6,7,8,9],
                   'num_specimen_seen': [0,1,2,3,4,5,6,7,8,9]},
                  )

data_b = data#resample(data, replace=True)
y = data_b['num_wings'].values
data_b.drop('num_wings', inplace=True, axis=1)
X = data_b.values
kf = KFold(n_splits=5, shuffle=True, random_state=0) 
j = 1

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in range(0,3):
        iter_val = "iter_"+str(j)
        j = j+1
        X_resample, y_resample = resample(X_train,y_train, replace=True)
        iters[iter_val] = {
           'X_train': X_resample,
           'X_test': X_test,
            'y_train': y_resample,
            'y_test': y_test   
        }   

        print(y_train, y_resample, y_test)
    print("**********************************")
# print("------------------------", iters)

In [ ]:

from sklearn.utils import resample

def bootstrap_val_old(final_data, amputed_col):
    # getting the size of the data and also storing index numbers so that we can use -->
    # list index to do resampling and reflect it on the actual data. This is less -->
    # complex than doing it directly on the data.
    data = final_data.copy()
    values = data.values
    data_index = list(range(0,27747))

    n_iterations = 10      
    n_samples = int(len(data_index) * 1)
    stats = list()

    # we want to do bootstrap validation for 100 iterations, so we will structure this as if it is 100 folds
    iters = {}

    for i in range(n_iterations):
        # dictionary item iter_0 ... iter_99
        iter_val = "iter_"+str(i)

        #resample and create a list with only index values
        train = resample(data_index, n_samples=n_samples)
        test = np.array([x for x in data_index if x not in train]) # time taking
        #will now generate the actual data samples using the indexes
        test_set = []
        train_set = []
        for j in test:
            test_set.append(values[j])
        for k in train:
            train_set.append(values[k])

        # creating dfs for easy manipulation, extract X and y
        new_cols = columns.copy()
        new_cols.append(amputed_col)
        train_df = pd.DataFrame(train_set, columns = new_cols)
        y_train = train_df.iloc[:, 46].values
        X_train = train_df.iloc[:, np.r_[0:46]].values

        test_df = pd.DataFrame(test_set, columns = new_cols)
        y_test = test_df.iloc[:, 46].values
        X_test = test_df.iloc[:, np.r_[0:46]].values
        iters[iter_val] = {
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test   
        }
    return iters



# Original Bootstrap Validation

In [ ]:
#One way to do this more efficiently is to use the difference method from the Index class in pandas.
#This method allows you to compute the set difference between the index of one dataframe and the
#index of another dataframe, which can be used to create the test set from the leftover observations
#in the resampled data.

#import datetime
from sklearn.utils import resample

def bootstrap_val_original(final_data, amputed_col):
    n_iterations = 100    
    
    # iters stores the train and test for each iteration of bootstrap sampling
    iters = {}
    #start = time.perf_counter()
    for i in range(n_iterations):
        # dictionary item iter_0 ... iter_99
        iter_val = "iter_"+str(i)

        #resample and create a list with only index values
        data = final_data.copy()
        y = data[amputed_col] # example dbp_ampute, this is the target with null values
        data.drop(amputed_col, inplace=True, axis=1)
        X = data
        
        X_resampled, y_resampled = resample(X, y, replace=True)
        
        X_test = X[~data.index.isin(X_resampled.index)]
        y_test = y[~data.index.isin(y_resampled.index)]
        
        iters[iter_val] = {
            'X_train': X_resampled.values,
            'X_test': X_test.values,
            'y_train': y_resampled.values,
            'y_test': y_test.values   
        }
    #end = time.perf_counter()
    #print(end - start)
    return iters

# Fixed Test Boostrap Validation

In [ ]:
# We split into test-train and then did sampling on the train
import datetime
from sklearn.utils import resample

def bootstrap_val(final_data, amputed_col):
    n_iterations = 100    
    
    # iters stores the train and test for each iteration of bootstrap sampling
    iters = {}

    for i in range(n_iterations):
        # dictionary item iter_0 ... iter_99
        iter_val = "iter_"+str(i)

        #resample and create a list with only index values
        data = final_data.copy()
        y = data[amputed_col] # example dbp_ampute, this is the target with null values
        data.drop(amputed_col, inplace=True, axis=1)
        X = data
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)
        
        X_train_r, y_train_r = resample(X_train, y_train)
        
        iters[iter_val] = {
            'X_train': X_train_r.values,
            'X_test': X_test.values,
            'y_train': y_train_r.values,
            'y_test': y_test.values   
        }
    return iters

# K Fold Bootstrap Validation

In [ ]:
# shuffle shuffle=True the data and divide into k folds (5) n_splits=5
# the k folds should have the same partitions, so we set random_state=0
# then we sample the train set in each fold 

from sklearn.model_selection import KFold
def k_folds_cross_bootstrap(final_data, k, n_bootstrap, amputed_col):
    iters = {}
    data = final_data.copy()
    y_data = data[amputed_col].values # example dbp_ampute, this is the target with null values
    data.drop(amputed_col, inplace=True, axis=1)
    X_data = data.values
    k_folds = KFold(n_splits = k, shuffle=True, random_state=0)
    
    i = 1
    for train_index, test_index in k_folds.split(X_data, y_data):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]
        for j in range(0,n_bootstrap):
            iter_val = "iter_"+str(i)
            X_resample, y_resample = resample(X_train,y_train, replace=True)
            iters[iter_val] = {
               'X_train': X_resample,
               'X_test': X_test,
                'y_train': y_resample,
                'y_test': y_test   
            }   
            i = i+1
    return iters

# Variables for bootstrap validation

In [ ]:
dbp_bv_5_perc = bootstrap_val(dbp_5, 'dbp_ampute')
dbp_bv_10_perc = bootstrap_val(dbp_10, 'dbp_ampute')
dbp_bv_20_perc = bootstrap_val(dbp_20, 'dbp_ampute')
dbp_bv_30_perc = bootstrap_val(dbp_30, 'dbp_ampute')
dbp_bv_40_perc = bootstrap_val(dbp_40, 'dbp_ampute')
dbp_bv_50_perc = bootstrap_val(dbp_50, 'dbp_ampute')

In [ ]:
dbp_bv_o_5_perc = bootstrap_val_original(dbp_5, 'dbp_ampute')
dbp_bv_o_10_perc = bootstrap_val_original(dbp_10, 'dbp_ampute')
dbp_bv_o_20_perc = bootstrap_val_original(dbp_20, 'dbp_ampute')
dbp_bv_o_30_perc = bootstrap_val_original(dbp_30, 'dbp_ampute')
dbp_bv_o_40_perc = bootstrap_val_original(dbp_40, 'dbp_ampute')
dbp_bv_o_50_perc = bootstrap_val_original(dbp_50, 'dbp_ampute')

In [ ]:
# k fold bootstrap variation (5 fold, 10 bv iterations)
dbp_5_fold_bv_5_perc = k_folds_cross_bootstrap(dbp_5, 5, 10, 'dbp_ampute')
dbp_5_fold_bv_10_perc = k_folds_cross_bootstrap(dbp_10, 5, 10, 'dbp_ampute')
dbp_5_fold_bv_20_perc = k_folds_cross_bootstrap(dbp_20, 5, 10, 'dbp_ampute')
dbp_5_fold_bv_30_perc = k_folds_cross_bootstrap(dbp_30, 5, 10, 'dbp_ampute')
dbp_5_fold_bv_40_perc = k_folds_cross_bootstrap(dbp_40, 5, 10, 'dbp_ampute')
dbp_5_fold_bv_50_perc = k_folds_cross_bootstrap(dbp_50, 5, 10, 'dbp_ampute')

In [ ]:
#SBP

In [ ]:
sbp_bv_5_perc = bootstrap_val(sbp_5, 'sbp_ampute')
sbp_bv_10_perc = bootstrap_val(sbp_10, 'sbp_ampute')
sbp_bv_20_perc = bootstrap_val(sbp_20, 'sbp_ampute')
sbp_bv_30_perc = bootstrap_val(sbp_30, 'sbp_ampute')
sbp_bv_40_perc = bootstrap_val(sbp_40, 'sbp_ampute')
sbp_bv_50_perc = bootstrap_val(sbp_50, 'sbp_ampute')

In [ ]:
sbp_bv_o_5_perc = bootstrap_val_original(sbp_5, 'sbp_ampute')
sbp_bv_o_10_perc = bootstrap_val_original(sbp_10, 'sbp_ampute')
sbp_bv_o_20_perc = bootstrap_val_original(sbp_20, 'sbp_ampute')
sbp_bv_o_30_perc = bootstrap_val_original(sbp_30, 'sbp_ampute')
sbp_bv_o_40_perc = bootstrap_val_original(sbp_40, 'sbp_ampute')
sbp_bv_o_50_perc = bootstrap_val_original(sbp_50, 'sbp_ampute')

In [ ]:
# k fold bootstrap variation (5 fold, 10 bv iterations)
sbp_5_fold_bv_5_perc = k_folds_cross_bootstrap(sbp_5, 5, 10, 'sbp_ampute')
sbp_5_fold_bv_10_perc = k_folds_cross_bootstrap(sbp_10, 5, 10, 'sbp_ampute')
sbp_5_fold_bv_20_perc = k_folds_cross_bootstrap(sbp_20, 5, 10, 'sbp_ampute')
sbp_5_fold_bv_30_perc = k_folds_cross_bootstrap(sbp_30, 5, 10, 'sbp_ampute')
sbp_5_fold_bv_40_perc = k_folds_cross_bootstrap(sbp_40, 5, 10, 'sbp_ampute')
sbp_5_fold_bv_50_perc = k_folds_cross_bootstrap(sbp_50, 5, 10, 'sbp_ampute')

In [ ]:
resp_bv_5_perc = bootstrap_val(resp_5, 'resp_ampute')
resp_bv_10_perc = bootstrap_val(resp_10, 'resp_ampute')
resp_bv_20_perc = bootstrap_val(resp_20, 'resp_ampute')
resp_bv_30_perc = bootstrap_val(resp_30, 'resp_ampute')
resp_bv_40_perc = bootstrap_val(resp_40, 'resp_ampute')
resp_bv_50_perc = bootstrap_val(resp_50, 'resp_ampute')

In [ ]:
resp_bv_o_5_perc = bootstrap_val_original(resp_5, 'resp_ampute')
resp_bv_o_10_perc = bootstrap_val_original(resp_10, 'resp_ampute')
resp_bv_o_20_perc = bootstrap_val_original(resp_20, 'resp_ampute')
resp_bv_o_30_perc = bootstrap_val_original(resp_30, 'resp_ampute')
resp_bv_o_40_perc = bootstrap_val_original(resp_40, 'resp_ampute')
resp_bv_o_50_perc = bootstrap_val_original(resp_50, 'resp_ampute')

In [ ]:
# k fold bootstrap variation (5 fold, 10 bv iterations)
resp_5_fold_bv_5_perc = k_folds_cross_bootstrap(resp_5, 5, 10, 'resp_ampute')
resp_5_fold_bv_10_perc = k_folds_cross_bootstrap(resp_10, 5, 10, 'resp_ampute')
resp_5_fold_bv_20_perc = k_folds_cross_bootstrap(resp_20, 5, 10, 'resp_ampute')
resp_5_fold_bv_30_perc = k_folds_cross_bootstrap(resp_30, 5, 10, 'resp_ampute')
resp_5_fold_bv_40_perc = k_folds_cross_bootstrap(resp_40, 5, 10, 'resp_ampute')
resp_5_fold_bv_50_perc = k_folds_cross_bootstrap(resp_50, 5, 10, 'resp_ampute')

In [ ]:
pulse_bv_5_perc = bootstrap_val(pulse_5, 'pulse_ampute')
pulse_bv_10_perc = bootstrap_val(pulse_10, 'pulse_ampute')
pulse_bv_20_perc = bootstrap_val(pulse_20, 'pulse_ampute')
pulse_bv_30_perc = bootstrap_val(pulse_30, 'pulse_ampute')
pulse_bv_40_perc = bootstrap_val(pulse_40, 'pulse_ampute')
pulse_bv_50_perc = bootstrap_val(pulse_50, 'pulse_ampute')

In [ ]:
pulse_bv_o_5_perc = bootstrap_val_original(pulse_5, 'pulse_ampute')
pulse_bv_o_10_perc = bootstrap_val_original(pulse_10, 'pulse_ampute')
pulse_bv_o_20_perc = bootstrap_val_original(pulse_20, 'pulse_ampute')
pulse_bv_o_30_perc = bootstrap_val_original(pulse_30, 'pulse_ampute')
pulse_bv_o_40_perc = bootstrap_val_original(pulse_40, 'pulse_ampute')
pulse_bv_o_50_perc = bootstrap_val_original(pulse_50, 'pulse_ampute')

In [ ]:
# k fold bootstrap variation (5 fold, 10 bv iterations)
pulse_5_fold_bv_5_perc = k_folds_cross_bootstrap(pulse_5, 5, 10, 'pulse_ampute')
pulse_5_fold_bv_10_perc = k_folds_cross_bootstrap(pulse_10, 5, 10, 'pulse_ampute')
pulse_5_fold_bv_20_perc = k_folds_cross_bootstrap(pulse_20, 5, 10, 'pulse_ampute')
pulse_5_fold_bv_30_perc = k_folds_cross_bootstrap(pulse_30, 5, 10, 'pulse_ampute')
pulse_5_fold_bv_40_perc = k_folds_cross_bootstrap(pulse_40, 5, 10, 'pulse_ampute')
pulse_5_fold_bv_50_perc = k_folds_cross_bootstrap(pulse_50, 5, 10, 'pulse_ampute')

# Variables for 5 fold cross validation

In [ ]:
folds_5_new = folds_5.copy()
dbp_5_folds_5_perc = k_folds_cross(dbp_5, 5, folds_5_new)
folds_5_new = folds_5.copy()
dbp_5_folds_10_perc = k_folds_cross(dbp_10, 5, folds_5_new)
folds_5_new = folds_5.copy()
dbp_5_folds_20_perc = k_folds_cross(dbp_20, 5, folds_5_new)
folds_5_new = folds_5.copy()
dbp_5_folds_30_perc = k_folds_cross(dbp_30, 5, folds_5_new)
folds_5_new = folds_5.copy()
dbp_5_folds_40_perc = k_folds_cross(dbp_40, 5, folds_5_new)
folds_5_new = folds_5.copy()
dbp_5_folds_50_perc = k_folds_cross(dbp_50, 5, folds_5_new)

In [ ]:
folds_5_new = folds_5.copy()
sbp_5_folds_5_perc = k_folds_cross(sbp_5, 5, folds_5_new)
folds_5_new = folds_5.copy()
sbp_5_folds_10_perc = k_folds_cross(sbp_10, 5, folds_5_new)
folds_5_new = folds_5.copy()
sbp_5_folds_20_perc = k_folds_cross(sbp_20, 5, folds_5_new)
folds_5_new = folds_5.copy()
sbp_5_folds_30_perc = k_folds_cross(sbp_30, 5, folds_5_new)
folds_5_new = folds_5.copy()
sbp_5_folds_40_perc = k_folds_cross(sbp_40, 5, folds_5_new)
folds_5_new = folds_5.copy()
sbp_5_folds_50_perc = k_folds_cross(sbp_50, 5, folds_5_new)

In [ ]:
folds_5_new = folds_5.copy()
resp_5_folds_5_perc = k_folds_cross(resp_5, 5, folds_5_new)
folds_5_new = folds_5.copy()
resp_5_folds_10_perc = k_folds_cross(resp_10, 5, folds_5_new)
folds_5_new = folds_5.copy()
resp_5_folds_20_perc = k_folds_cross(resp_20, 5, folds_5_new)
folds_5_new = folds_5.copy()
resp_5_folds_30_perc = k_folds_cross(resp_30, 5, folds_5_new)
folds_5_new = folds_5.copy()
resp_5_folds_40_perc = k_folds_cross(resp_40, 5, folds_5_new)
folds_5_new = folds_5.copy()
resp_5_folds_50_perc = k_folds_cross(resp_50, 5, folds_5_new)

In [ ]:
folds_5_new = folds_5.copy()
pulse_5_folds_5_perc = k_folds_cross(pulse_5, 5, folds_5_new)
folds_5_new = folds_5.copy()
pulse_5_folds_10_perc = k_folds_cross(pulse_10, 5, folds_5_new)
folds_5_new = folds_5.copy()
pulse_5_folds_20_perc = k_folds_cross(pulse_20, 5, folds_5_new)
folds_5_new = folds_5.copy()
pulse_5_folds_30_perc = k_folds_cross(pulse_30, 5, folds_5_new)
folds_5_new = folds_5.copy()
pulse_5_folds_40_perc = k_folds_cross(pulse_40, 5, folds_5_new)
folds_5_new = folds_5.copy()
pulse_5_folds_50_perc = k_folds_cross(pulse_50, 5, folds_5_new)

# Variables for 10 fold cross Validation

In [ ]:
folds_10_new = folds_10.copy()
dbp_10_folds_5_perc = k_folds_cross(dbp_5, 10, folds_10_new)
folds_10_new = folds_10.copy()
dbp_10_folds_10_perc = k_folds_cross(dbp_10, 10, folds_10_new)
folds_10_new = folds_10.copy()
dbp_10_folds_20_perc = k_folds_cross(dbp_20, 10, folds_10_new)
folds_10_new = folds_10.copy()
dbp_10_folds_30_perc = k_folds_cross(dbp_30, 10, folds_10_new)
folds_10_new = folds_10.copy()
dbp_10_folds_40_perc = k_folds_cross(dbp_40, 10, folds_10_new)
folds_10_new = folds_10.copy()
dbp_10_folds_50_perc = k_folds_cross(dbp_50, 10, folds_10_new)

In [ ]:
folds_10_new = folds_10.copy()
sbp_10_folds_5_perc = k_folds_cross(sbp_5, 10, folds_10_new)
folds_10_new = folds_10.copy()
sbp_10_folds_10_perc = k_folds_cross(sbp_10, 10, folds_10_new)
folds_10_new = folds_10.copy()
sbp_10_folds_20_perc = k_folds_cross(sbp_20, 10, folds_10_new)
folds_10_new = folds_10.copy()
sbp_10_folds_30_perc = k_folds_cross(sbp_30, 10, folds_10_new)
folds_10_new = folds_10.copy()
sbp_10_folds_40_perc = k_folds_cross(sbp_40, 10, folds_10_new)
folds_10_new = folds_10.copy()
sbp_10_folds_50_perc = k_folds_cross(sbp_50, 10, folds_10_new)

In [ ]:
folds_10_new = folds_10.copy()
resp_10_folds_5_perc = k_folds_cross(resp_5, 10, folds_10_new)
folds_10_new = folds_10.copy()
resp_10_folds_10_perc = k_folds_cross(resp_10, 10, folds_10_new)
folds_10_new = folds_10.copy()
resp_10_folds_20_perc = k_folds_cross(resp_20, 10, folds_10_new)
folds_10_new = folds_10.copy()
resp_10_folds_30_perc = k_folds_cross(resp_30, 10, folds_10_new)
folds_10_new = folds_10.copy()
resp_10_folds_40_perc = k_folds_cross(resp_40, 10, folds_10_new)
folds_10_new = folds_10.copy()
resp_10_folds_50_perc = k_folds_cross(resp_50, 10, folds_10_new)

In [ ]:
folds_10_new = folds_10.copy()
pulse_10_folds_5_perc = k_folds_cross(pulse_5, 10, folds_10_new)
folds_10_new = folds_10.copy()
pulse_10_folds_10_perc = k_folds_cross(pulse_10, 10, folds_10_new)
folds_10_new = folds_10.copy()
pulse_10_folds_20_perc = k_folds_cross(pulse_20, 10, folds_10_new)
folds_10_new = folds_10.copy()
pulse_10_folds_30_perc = k_folds_cross(pulse_30, 10, folds_10_new)
folds_10_new = folds_10.copy()
pulse_10_folds_40_perc = k_folds_cross(pulse_40, 10, folds_10_new)
folds_10_new = folds_10.copy()
pulse_10_folds_50_perc = k_folds_cross(pulse_50, 10, folds_10_new)

# Mean/Median 

In [ ]:
def mean_median(folds, target, metric):
    length = []
    MAE = []
    MSE = []
    # the train set should exclude missing values of the target
    folds_new = {}
    for fold in folds:
        folds_new[fold] = {}
        # extract train sets from the folds
        X_train = folds[fold]['X_train']
#         print(type(X_train))
        X_train_copy = X_train.copy()
        y_train = folds[fold]['y_train']
        y_train_copy = y_train.copy()

        # extract test sets from the folds
        X_test = folds[fold]['X_test']
        X_test_copy = X_test.copy()
        y_test = folds[fold]['y_test']
        y_test_copy = y_test.copy()
        
        # collecting the null values in target's indices
        train_indices_null = []
        for i,y in enumerate(y_train_copy):
            if np.isnan(y):
                train_indices_null.append(i)
        # removing all the null values from training target and corresponding features set
        y_train_copy = np.delete(y_train_copy, train_indices_null, 0)
        X_train_copy = np.delete(X_train_copy, train_indices_null, 0)
        folds_new[fold]['X_train'] = X_train_copy
        folds_new[fold]['y_train'] = y_train_copy
        if metric == 'mean':
            mean_median = y_train_copy.mean()
        else:
            mean_median = np.median(y_train_copy)# y_train_copy.median()
        # we want to only predict the missing values, so our new test set will be the values that are null in target

        # collecting the null values in target's indices
        test_indices_null = []
        for i,y in enumerate(y_test_copy):
            if not np.isnan(y):
                test_indices_null.append(i)
        # keeping only the null values from test target and corresponding features set
        y_test_copy = np.delete(y_test_copy, test_indices_null, 0)
        X_test_copy = np.delete(X_test_copy, test_indices_null, 0)
        folds_new[fold]['X_test'] = X_test_copy
        folds_new[fold]['y_test'] = y_test_copy
        
        
        # saving the original y values
        dbp_final_df = pd.DataFrame(X_test_copy, columns = columns)
        dbp_final_df = dbp_final_df[[target]]
        y_test = dbp_final_df
        
#         mae_value = mae(y_test, y_test)#y_train_copy.mean())
        mean_list = [mean_median] * len(y_test)
        mae_value=metrics.mean_absolute_error(y_test, mean_list)# y_train_copy.mean())
        mse=metrics.mean_squared_error(y_test, mean_list)
        MAE.append(mae_value)
        MSE.append(mse)
#         mse_value = mean_squared_error(actual, pred)
#         print("mae-",mae_value)

    return np.mean(MAE), np.std(MAE), np.mean(MSE), np.std(MSE)


# ML methods on data with missing values

In [ ]:
def train_test_wo_null(folds, model, target):
    length = []
    MAE = []
    MSE = []
    # the train set should exclude missing values of the target
    folds_new = {}
    for fold in folds:
        folds_new[fold] = {}
        # extract train sets from the folds
        X_train = folds[fold]['X_train']
        X_train_copy = X_train.copy()
        y_train = folds[fold]['y_train']
        y_train_copy = y_train.copy()

        # extract test sets from the folds
        X_test = folds[fold]['X_test']
        X_test_copy = X_test.copy()
        y_test = folds[fold]['y_test']
        y_test_copy = y_test.copy()

        # collecting the null values in target's indices
        train_indices_null = []
        for i,y in enumerate(y_train_copy):
            if np.isnan(y):
                train_indices_null.append(i)
        # removing all the null values from training target and corresponding features set
        y_train_copy = np.delete(y_train_copy, train_indices_null, 0)
        X_train_copy = np.delete(X_train_copy, train_indices_null, 0)
        folds_new[fold]['X_train'] = X_train_copy
        folds_new[fold]['y_train'] = y_train_copy

        # we want to only predict the missing values, so our new test set will be the values that are null in target

        # collecting the null values in target's indices
        test_indices_null = []
        for i,y in enumerate(y_test_copy):
            if not np.isnan(y):
                test_indices_null.append(i)
        # keeping only the null values from test target and corresponding features set
        y_test_copy = np.delete(y_test_copy, test_indices_null, 0)
        X_test_copy = np.delete(X_test_copy, test_indices_null, 0)
        folds_new[fold]['X_test'] = X_test_copy
        folds_new[fold]['y_test'] = y_test_copy
        
        # we kept the amputed values in the original columns, so we will remove this before training
        X_train_copy_df = pd.DataFrame(X_train_copy, columns = columns)
        X_test_copy_df = pd.DataFrame(X_test_copy, columns = columns)
        X_train_copy_df = X_train_copy_df.drop(target, axis=1)
        X_test_copy_df = X_test_copy_df.drop(target, axis=1)
        
        # saving the original y values
        dbp_final_df = pd.DataFrame(X_test_copy, columns = columns)
        dbp_final_df = dbp_final_df[[target]]
        y_test = dbp_final_df
        length.append(len(y_test))
        model.fit(X_train_copy_df, y_train_copy)
        y_pred = model.predict(X_test_copy_df)
        score = model.score(X_test_copy_df, y_test)
        a=metrics.mean_absolute_error(y_test, y_pred)
        mse=metrics.mean_squared_error(y_test, y_pred)
        MAE.append(a)
        MSE.append(mse)
    print("Length-", np.mean(length))
    return np.mean(MAE), np.std(MAE), np.mean(MSE), np.std(MSE)
        

# Diastolic Blood Pressure 

In [ ]:
# DBP 5 fold cross Validation MEAN

In [ ]:
five_fold_mean_MAE_mean = []
five_fold_mean_MAE_std = []
five_fold_mean_MSE_mean = []
five_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_5_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_10_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_20_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_30_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_40_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_50_perc, 'dbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

dbp_5_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_mean_MAE_mean,
                   'MAE_std':five_fold_mean_MAE_std,
                   'MSE_mean': five_fold_mean_MSE_mean,
                   'MSE_std':five_fold_mean_MSE_std}
                  )
dbp_5_fold_mean.style.set_caption("DBP - 5 Fold Cross Validation - MEAN")

In [ ]:
# 5 fold cross Validation MEDIAN

In [ ]:

five_fold_median_MAE_mean = []
five_fold_median_MAE_std = []
five_fold_median_MSE_mean = []
five_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_5_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_10_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_20_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_30_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_40_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_folds_50_perc, 'dbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

dbp_5_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_median_MAE_mean,
                   'MAE_std':five_fold_median_MAE_std,
                   'MSE_mean': five_fold_median_MSE_mean,
                   'MSE_std':five_fold_median_MSE_std}
                  )
dbp_5_fold_median.style.set_caption("DBP - 5 Fold Cross Validation - MEDIAN")

In [ ]:
# 5 fold cv Linear

In [ ]:
five_fold_Lin_MAE_mean = []
five_fold_Lin_MAE_std = []
five_fold_Lin_MSE_mean = []
five_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_5_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_10_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_20_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_30_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_40_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_50_perc, linear_model, 'dbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

dbp_5_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_Lin_MAE_mean,
                   'MAE_std':five_fold_Lin_MAE_std,
                   'MSE_mean': five_fold_Lin_MSE_mean,
                   'MSE_std':five_fold_Lin_MSE_std}
                  )
dbp_5_fold_linear.style.set_caption("DBP - 5 Fold Cross Validation - LINEAR REGRESSION")

In [ ]:
# 5 fold cross Validation RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

five_fold_RF_MAE_mean = []
five_fold_RF_MAE_std = []
five_fold_RF_MSE_mean = []
five_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_5_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_10_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_20_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_30_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_40_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_50_perc, random_forest_model, 'dbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

dbp_5_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_RF_MAE_mean,
                   'MAE_std':five_fold_RF_MAE_std,
                   'MSE_mean': five_fold_RF_MSE_mean,
                   'MSE_std':five_fold_RF_MSE_std}
                  )
dbp_5_fold_random_forest.style.set_caption("DBP - 5 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
# 5 fold cross validation XGboost dbp

In [ ]:
import warnings;
warnings.filterwarnings('ignore');

In [ ]:
five_fold_XGB_MAE_mean = []
five_fold_XGB_MAE_std = []
five_fold_XGB_MSE_mean = []
five_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_5_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_10_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_20_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_30_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_40_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_50_perc, xgboost_model, 'dbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

dbp_5_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_XGB_MAE_mean,
                   'MAE_std':five_fold_XGB_MAE_std,
                   'MSE_mean': five_fold_XGB_MSE_mean,
                   'MSE_std':five_fold_XGB_MSE_std}
                  )
dbp_5_fold_XGB.style.set_caption("DBP - 5 Fold Cross Validation - BOOSTING")

In [ ]:
five_fold_BAG_MAE_mean = []
five_fold_BAG_MAE_std = []
five_fold_BAG_MSE_mean = []
five_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_5_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_10_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_20_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_30_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_40_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_folds_50_perc, bagging_model, 'dbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)

dbp_5_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_BAG_MAE_mean,
                   'MAE_std':five_fold_BAG_MAE_std,
                   'MSE_mean': five_fold_BAG_MSE_mean,
                   'MSE_std':five_fold_BAG_MSE_std}
                  )
dbp_5_fold_BAG.style.set_caption("DBP - 5 Fold Cross Validation - BAGGING")

In [ ]:
folds_5_new = folds_5.copy()
dbp_5_folds_50_perc = k_folds_cross(dbp_50, 5, folds_5_new)
train_test_wo_null(dbp_5_folds_50_perc, RandomForestRegressor(n_estimators=20, random_state=0), 'dbp_final')

In [ ]:
# 10 fold cross validation - MEAN dbp

In [ ]:
ten_fold_mean_MAE_mean = []
ten_fold_mean_MAE_std = []
ten_fold_mean_MSE_mean = []
ten_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_5_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_10_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_20_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_30_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_40_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_50_perc, 'dbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

dbp_10_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_mean_MAE_mean,
                   'MAE_std':ten_fold_mean_MAE_std,
                   'MSE_mean': ten_fold_mean_MSE_mean,
                   'MSE_std':ten_fold_mean_MSE_std}
                  )
dbp_10_fold_mean.style.set_caption("DBP - 10 Fold Cross Validation - MEAN")

In [ ]:
ten_fold_median_MAE_mean = []
ten_fold_median_MAE_std = []
ten_fold_median_MSE_mean = []
ten_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_5_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_10_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_20_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_30_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_40_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_10_folds_50_perc, 'dbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

dbp_10_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_median_MAE_mean,
                   'MAE_std':ten_fold_median_MAE_std,
                   'MSE_mean': ten_fold_median_MSE_mean,
                   'MSE_std':ten_fold_median_MSE_std}
                  )
dbp_10_fold_median.style.set_caption("DBP - 10 Fold Cross Validation - MEDIAN")

In [ ]:
ten_fold_Lin_MAE_mean = []
ten_fold_Lin_MAE_std = []
ten_fold_Lin_MSE_mean = []
ten_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_5_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_10_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_20_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_30_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_40_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_50_perc, linear_model, 'dbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

dbp_10_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_Lin_MAE_mean,
                   'MAE_std':ten_fold_Lin_MAE_std,
                   'MSE_mean': ten_fold_Lin_MSE_mean,
                   'MSE_std':ten_fold_Lin_MSE_std}
                  )
dbp_10_fold_linear.style.set_caption("DBP - 10 Fold Cross Validation - LINEAR REGRESSION")

In [ ]:
ten_fold_RF_MAE_mean = []
ten_fold_RF_MAE_std = []
ten_fold_RF_MSE_mean = []
ten_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_5_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_10_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_20_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_30_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_40_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_50_perc, random_forest_model, 'dbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

dbp_10_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_RF_MAE_mean,
                   'MAE_std':ten_fold_RF_MAE_std,
                   'MSE_mean': ten_fold_RF_MSE_mean,
                   'MSE_std':ten_fold_RF_MSE_std}
                  )
dbp_10_fold_random_forest.style.set_caption("DBP - 10 Fold Cross Validation - RANDOM FOREST REGRESSION")

In [ ]:
ten_fold_XGB_MAE_mean = []
ten_fold_XGB_MAE_std = []
ten_fold_XGB_MSE_mean = []
ten_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_5_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_10_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_20_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_30_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_40_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_50_perc, xgboost_model, 'dbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

dbp_10_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_XGB_MAE_mean,
                   'MAE_std':ten_fold_XGB_MAE_std,
                   'MSE_mean': ten_fold_XGB_MSE_mean,
                   'MSE_std':ten_fold_XGB_MSE_std}
                  )
dbp_10_fold_XGB.style.set_caption("DBP - 10 Fold Cross Validation - BOOSTING")

In [ ]:
ten_fold_BAG_MAE_mean = []
ten_fold_BAG_MAE_std = []
ten_fold_BAG_MSE_mean = []
ten_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_5_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_10_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_20_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_30_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_40_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_10_folds_50_perc, bagging_model, 'dbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)

dbp_10_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_BAG_MAE_mean,
                   'MAE_std':ten_fold_BAG_MAE_std,
                   'MSE_mean': ten_fold_BAG_MSE_mean,
                   'MSE_std':ten_fold_BAG_MSE_std}
                  )
dbp_10_fold_BAG.style.set_caption("DBP - 10 Fold Cross Validation - BAGGING")

In [ ]:
# dbp BV mean

In [ ]:
BV_mean_MAE_mean = []
BV_mean_MAE_std = []
BV_mean_MSE_mean = []
BV_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_5_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_10_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_20_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_30_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_40_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_50_perc, 'dbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

dbp_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_mean_MAE_mean,
                   'MAE_std':BV_mean_MAE_std,
                   'MSE_mean': BV_mean_MSE_mean,
                   'MSE_std':BV_mean_MSE_std}
                  )
dbp_bv_mean.style.set_caption("DBP - Bootstrap Validation - MEAN")

In [ ]:
# Original bootstrap method
BV_O_mean_MAE_mean = []
BV_O_mean_MAE_std = []
BV_O_mean_MSE_mean = []
BV_O_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_5_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_10_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_20_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_30_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_40_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_50_perc, 'dbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

dbp_bv_o_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_mean_MAE_mean,
                   'MAE_std':BV_O_mean_MAE_std,
                   'MSE_mean': BV_O_mean_MSE_mean,
                   'MSE_std':BV_O_mean_MSE_std}
                  )
dbp_bv_o_mean.style.set_caption("DBP - Bootstrap Validation (original)- MEAN")

In [ ]:
# k fold bootstrap method
BV_5_fold_mean_MAE_mean = []
BV_5_fold_mean_MAE_std = []
BV_5_fold_mean_MSE_mean = []
BV_5_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_5_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_10_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_20_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_30_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_40_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_50_perc, 'dbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

dbp_5_fold_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_mean_MAE_mean,
                   'MAE_std':BV_5_fold_mean_MAE_std,
                   'MSE_mean': BV_5_fold_mean_MSE_mean,
                   'MSE_std':BV_5_fold_mean_MSE_std}
                  )
dbp_5_fold_bv_mean.style.set_caption("DBP - Bootstrap Validation K fold - MEAN")

In [ ]:
# dbp boostrap validation median

In [ ]:
BV_median_MAE_mean = []
BV_median_MAE_std = []
BV_median_MSE_mean = []
BV_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_5_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_10_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_20_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_30_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_40_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_50_perc, 'dbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

dbp_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_median_MAE_mean,
                   'MAE_std':BV_median_MAE_std,
                   'MSE_mean': BV_median_MSE_mean,
                   'MSE_std':BV_median_MSE_std}
                  )
dbp_bv_median.style.set_caption("DBP - Bootstrap Validation - MEDIAN")

In [ ]:
BV_O_median_MAE_mean = []
BV_O_median_MAE_std = []
BV_O_median_MSE_mean = []
BV_O_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_5_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_10_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_20_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_30_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_40_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_bv_o_50_perc, 'dbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

dbp_bv_o_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_median_MAE_mean,
                   'MAE_std':BV_O_median_MAE_std,
                   'MSE_mean': BV_O_median_MSE_mean,
                   'MSE_std':BV_O_median_MSE_std}
                  )
dbp_bv_o_median.style.set_caption("DBP - Bootstrap Validation(Original) - MEDIAN")

In [ ]:
BV_5_fold_median_MAE_mean = []
BV_5_fold_median_MAE_std = []
BV_5_fold_median_MSE_mean = []
BV_5_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_5_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_10_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_20_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_30_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_40_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(dbp_5_fold_bv_50_perc, 'dbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

dbp_5_fold_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_median_MAE_mean,
                   'MAE_std':BV_5_fold_median_MAE_std,
                   'MSE_mean': BV_5_fold_median_MSE_mean,
                   'MSE_std':BV_5_fold_median_MSE_std}
                  )
dbp_5_fold_bv_median.style.set_caption("DBP - Bootstrap Validation K fold - MEDIAN")

In [ ]:
BV_Lin_MAE_mean = []
BV_Lin_MAE_std = []
BV_Lin_MSE_mean = []
BV_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_5_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_10_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_20_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_30_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_40_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_50_perc, linear_model, 'dbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

dbp_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_Lin_MAE_mean,
                   'MAE_std':BV_Lin_MAE_std,
                   'MSE_mean': BV_Lin_MSE_mean,
                   'MSE_std':BV_Lin_MSE_std}
                  )
dbp_bv_linear.style.set_caption("DBP - Bootstrap Validation - LINEAR REGRESSION")

In [ ]:
BV_O_Lin_MAE_mean = []
BV_O_Lin_MAE_std = []
BV_O_Lin_MSE_mean = []
BV_O_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_5_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_10_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_20_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_30_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_40_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_50_perc, linear_model, 'dbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

dbp_bv_o_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_Lin_MAE_mean,
                   'MAE_std':BV_O_Lin_MAE_std,
                   'MSE_mean': BV_O_Lin_MSE_mean,
                   'MSE_std':BV_O_Lin_MSE_std}
                  )
dbp_bv_o_linear.style.set_caption("DBP - Bootstrap Validation(Original) - LINEAR REGRESSION")

In [ ]:
BV_5_fold_Lin_MAE_mean = []
BV_5_fold_Lin_MAE_std = []
BV_5_fold_Lin_MSE_mean = []
BV_5_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_5_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_10_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_20_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_30_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_40_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_50_perc, linear_model, 'dbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

dbp_5_fold_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_Lin_MAE_mean,
                   'MAE_std':BV_5_fold_Lin_MAE_std,
                   'MSE_mean': BV_5_fold_Lin_MSE_mean,
                   'MSE_std':BV_5_fold_Lin_MSE_std}
                  )
dbp_5_fold_bv_linear.style.set_caption("DBP - Bootstrap Validation K fold - LINEAR REGRESSION")

In [ ]:
#boostrap validation random forest

In [ ]:
BV_RF_MAE_mean = []
BV_RF_MAE_std = []
BV_RF_MSE_mean = []
BV_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_5_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_10_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_20_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_30_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_40_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_50_perc, random_forest_model, 'dbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

dbp_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_RF_MAE_mean,
                   'MAE_std':BV_RF_MAE_std,
                   'MSE_mean': BV_RF_MSE_mean,
                   'MSE_std':BV_RF_MSE_std}
                  )
dbp_bv_random_forest.style.set_caption("DBP - Bootstrap Validation - RANDOM FOREST REGRESSION")

In [ ]:
BV_O_RF_MAE_mean = []
BV_O_RF_MAE_std = []
BV_O_RF_MSE_mean = []
BV_O_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_5_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_10_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_20_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_30_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_40_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_50_perc, random_forest_model, 'dbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

dbp_bv_o_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_RF_MAE_mean,
                   'MAE_std':BV_O_RF_MAE_std,
                   'MSE_mean': BV_O_RF_MSE_mean,
                   'MSE_std':BV_O_RF_MSE_std}
                  )
dbp_bv_o_random_forest.style.set_caption("DBP - Bootstrap Validation(Original) - RANDOM FOREST REGRESSION")

In [ ]:
BV_5_fold_RF_MAE_mean = []
BV_5_fold_RF_MAE_std = []
BV_5_fold_RF_MSE_mean = []
BV_5_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_5_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_10_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_20_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_30_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_40_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_50_perc, random_forest_model, 'dbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

dbp_5_fold_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_RF_MAE_mean,
                   'MAE_std':BV_5_fold_RF_MAE_std,
                   'MSE_mean': BV_5_fold_RF_MSE_mean,
                   'MSE_std':BV_5_fold_RF_MSE_std}
                  )
dbp_5_fold_bv_random_forest.style.set_caption("DBP - Bootstrap Validation K fold - RANDOM FOREST REGRESSION")

In [ ]:
BV_XGB_MAE_mean = []
BV_XGB_MAE_std = []
BV_XGB_MSE_mean = []
BV_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_5_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_10_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_20_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_30_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_40_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_50_perc, xgboost_model, 'dbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

dbp_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_XGB_MAE_mean,
                   'MAE_std':BV_XGB_MAE_std,
                   'MSE_mean': BV_XGB_MSE_mean,
                   'MSE_std':BV_XGB_MSE_std}
                  )
dbp_bv_XGB.style.set_caption("DBP - Bootstrap Validation - BOOSTING")

In [ ]:
BV_O_XGB_MAE_mean = []
BV_O_XGB_MAE_std = []
BV_O_XGB_MSE_mean = []
BV_O_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_5_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_10_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_20_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_30_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_40_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_50_perc, xgboost_model, 'dbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

dbp_bv_o_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_XGB_MAE_mean,
                   'MAE_std':BV_O_XGB_MAE_std,
                   'MSE_mean': BV_O_XGB_MSE_mean,
                   'MSE_std':BV_O_XGB_MSE_std}
                  )
dbp_bv_o_XGB.style.set_caption("DBP - Bootstrap Validation(Original) - BOOSTING")

In [ ]:
BV_5_fold_XGB_MAE_mean = []
BV_5_fold_XGB_MAE_std = []
BV_5_fold_XGB_MSE_mean = []
BV_5_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_5_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_10_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_20_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_30_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_40_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_50_perc, xgboost_model, 'dbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

dbp_5_fold_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_XGB_MAE_mean,
                   'MAE_std':BV_5_fold_XGB_MAE_std,
                   'MSE_mean': BV_5_fold_XGB_MSE_mean,
                   'MSE_std':BV_5_fold_XGB_MSE_std}
                  )
dbp_5_fold_bv_XGB.style.set_caption("DBP - Bootstrap Validation K fold - BOOSTING")

In [ ]:
BV_BAG_MAE_mean = []
BV_BAG_MAE_std = []
BV_BAG_MSE_mean = []
BV_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_5_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_10_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_20_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_30_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_40_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_50_perc, bagging_model, 'dbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)

dbp_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_BAG_MAE_mean,
                   'MAE_std':BV_BAG_MAE_std,
                   'MSE_mean': BV_BAG_MSE_mean,
                   'MSE_std':BV_BAG_MSE_std}
                  )
dbp_bv_BAG.style.set_caption("DBP - Bootstrap Validation - BAGGING")

In [ ]:
BV_O_BAG_MAE_mean = []
BV_O_BAG_MAE_std = []
BV_O_BAG_MSE_mean = []
BV_O_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_5_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_10_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_20_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_30_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_40_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_bv_o_50_perc, bagging_model, 'dbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)

dbp_bv_o_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_BAG_MAE_mean,
                   'MAE_std':BV_O_BAG_MAE_std,
                   'MSE_mean': BV_O_BAG_MSE_mean,
                   'MSE_std':BV_O_BAG_MSE_std}
                  )
dbp_bv_o_BAG.style.set_caption("DBP - Bootstrap Validation(Original) - BAGGING")

In [ ]:
BV_5_fold_BAG_MAE_mean = []
BV_5_fold_BAG_MAE_std = []
BV_5_fold_BAG_MSE_mean = []
BV_5_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_5_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_10_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_20_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_30_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_40_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(dbp_5_fold_bv_50_perc, bagging_model, 'dbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)

dbp_5_fold_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_BAG_MAE_mean,
                   'MAE_std':BV_5_fold_BAG_MAE_std,
                   'MSE_mean': BV_5_fold_BAG_MSE_mean,
                   'MSE_std':BV_5_fold_BAG_MSE_std}
                  )
dbp_5_fold_bv_BAG.style.set_caption("DBP - Bootstrap Validation K fold - BAGGING")

In [ ]:
dbp_bv_BAG

In [ ]:
dbp_bv_BAG_temp = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': [1.809592, 1.852366, 1.821345, 1.862406, 1.903937, 1.971027],
                   }
                  )
dbp_bv_BAG_temp.style.set_caption("DBP - Bootstrap Validation - BAGGING")


# Systolic Blood Pressure

In [ ]:
five_fold_mean_MAE_mean = []
five_fold_mean_MAE_std = []
five_fold_mean_MSE_mean = []
five_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_5_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_10_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_20_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_30_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_40_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_50_perc, 'sbp_final', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

sbp_5_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_mean_MAE_mean,
                   'MAE_std':five_fold_mean_MAE_std,
                   'MSE_mean': five_fold_mean_MSE_mean,
                   'MSE_std':five_fold_mean_MSE_std}
                  )
sbp_5_fold_mean.style.set_caption("sbp - 5 Fold Cross Validation - MEAN")

In [ ]:

five_fold_median_MAE_mean = []
five_fold_median_MAE_std = []
five_fold_median_MSE_mean = []
five_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_5_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_10_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_20_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_30_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_40_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_folds_50_perc, 'sbp_final', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

sbp_5_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_median_MAE_mean,
                   'MAE_std':five_fold_median_MAE_std,
                   'MSE_mean': five_fold_median_MSE_mean,
                   'MSE_std':five_fold_median_MSE_std}
                  )
sbp_5_fold_median.style.set_caption("sbp - 5 Fold Cross Validation - MEDIAN")

In [ ]:
five_fold_Lin_MAE_mean = []
five_fold_Lin_MAE_std = []
five_fold_Lin_MSE_mean = []
five_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_5_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_10_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_20_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_30_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_40_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_50_perc, linear_model, 'sbp_final')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

sbp_5_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_Lin_MAE_mean,
                   'MAE_std':five_fold_Lin_MAE_std,
                   'MSE_mean': five_fold_Lin_MSE_mean,
                   'MSE_std':five_fold_Lin_MSE_std}
                  )
sbp_5_fold_linear.style.set_caption("sbp - 5 Fold Cross Validation - LINEAR REGRESSION")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

five_fold_RF_MAE_mean = []
five_fold_RF_MAE_std = []
five_fold_RF_MSE_mean = []
five_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_5_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_10_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_20_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_30_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_40_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_50_perc, random_forest_model, 'sbp_final')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

sbp_5_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_RF_MAE_mean,
                   'MAE_std':five_fold_RF_MAE_std,
                   'MSE_mean': five_fold_RF_MSE_mean,
                   'MSE_std':five_fold_RF_MSE_std}
                  )
sbp_5_fold_random_forest.style.set_caption("sbp - 5 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
five_fold_XGB_MAE_mean = []
five_fold_XGB_MAE_std = []
five_fold_XGB_MSE_mean = []
five_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_5_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_10_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_20_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_30_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_40_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_50_perc, xgboost_model, 'sbp_final')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

sbp_5_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_XGB_MAE_mean,
                   'MAE_std':five_fold_XGB_MAE_std,
                   'MSE_mean': five_fold_XGB_MSE_mean,
                   'MSE_std':five_fold_XGB_MSE_std}
                  )
sbp_5_fold_XGB.style.set_caption("sbp - 5 Fold Cross Validation - BOOSTING")

In [ ]:
five_fold_BAG_MAE_mean = []
five_fold_BAG_MAE_std = []
five_fold_BAG_MSE_mean = []
five_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_5_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_10_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_20_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_30_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_40_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_folds_50_perc, bagging_model, 'sbp_final')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)

sbp_5_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_BAG_MAE_mean,
                   'MAE_std':five_fold_BAG_MAE_std,
                   'MSE_mean': five_fold_BAG_MSE_mean,
                   'MSE_std':five_fold_BAG_MSE_std}
                  )
sbp_5_fold_BAG.style.set_caption("sbp - 5 Fold Cross Validation - BAGGING")

In [ ]:
ten_fold_mean_MAE_mean = []
ten_fold_mean_MAE_std = []
ten_fold_mean_MSE_mean = []
ten_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_5_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_10_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_20_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_30_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_40_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_50_perc, 'sbp_final', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

sbp_10_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_mean_MAE_mean,
                   'MAE_std':ten_fold_mean_MAE_std,
                   'MSE_mean': ten_fold_mean_MSE_mean,
                   'MSE_std':ten_fold_mean_MSE_std}
                  )
sbp_10_fold_mean.style.set_caption("sbp - 10 Fold Cross Validation - MEAN")

In [ ]:
ten_fold_median_MAE_mean = []
ten_fold_median_MAE_std = []
ten_fold_median_MSE_mean = []
ten_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_5_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_10_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_20_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_30_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_40_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_10_folds_50_perc, 'sbp_final', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

sbp_10_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_median_MAE_mean,
                   'MAE_std':ten_fold_median_MAE_std,
                   'MSE_mean': ten_fold_median_MSE_mean,
                   'MSE_std':ten_fold_median_MSE_std}
                  )
sbp_10_fold_median.style.set_caption("sbp - 10 Fold Cross Validation - MEDIAN")

In [ ]:
ten_fold_Lin_MAE_mean = []
ten_fold_Lin_MAE_std = []
ten_fold_Lin_MSE_mean = []
ten_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_5_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_10_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_20_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_30_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_40_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_50_perc, linear_model, 'sbp_final')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

sbp_10_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_Lin_MAE_mean,
                   'MAE_std':ten_fold_Lin_MAE_std,
                   'MSE_mean': ten_fold_Lin_MSE_mean,
                   'MSE_std':ten_fold_Lin_MSE_std}
                  )
sbp_10_fold_linear.style.set_caption("sbp - 10 Fold Cross Validation - LINEAR REGRESSION")

In [ ]:
ten_fold_RF_MAE_mean = []
ten_fold_RF_MAE_std = []
ten_fold_RF_MSE_mean = []
ten_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_5_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_10_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_20_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_30_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_40_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_50_perc, random_forest_model, 'sbp_final')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

sbp_10_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_RF_MAE_mean,
                   'MAE_std':ten_fold_RF_MAE_std,
                   'MSE_mean': ten_fold_RF_MSE_mean,
                   'MSE_std':ten_fold_RF_MSE_std}
                  )
sbp_10_fold_random_forest.style.set_caption("sbp - 10 Fold Cross Validation - RANDOM FOREST REGRESSION")

In [ ]:
ten_fold_XGB_MAE_mean = []
ten_fold_XGB_MAE_std = []
ten_fold_XGB_MSE_mean = []
ten_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_5_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_10_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_20_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_30_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_40_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_50_perc, xgboost_model, 'sbp_final')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

sbp_10_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_XGB_MAE_mean,
                   'MAE_std':ten_fold_XGB_MAE_std,
                   'MSE_mean': ten_fold_XGB_MSE_mean,
                   'MSE_std':ten_fold_XGB_MSE_std}
                  )
sbp_10_fold_XGB.style.set_caption("sbp - 10 Fold Cross Validation - BOOSTING")

In [ ]:
ten_fold_BAG_MAE_mean = []
ten_fold_BAG_MAE_std = []
ten_fold_BAG_MSE_mean = []
ten_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_5_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_10_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_20_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_30_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_40_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_10_folds_50_perc, bagging_model, 'sbp_final')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)

sbp_10_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_BAG_MAE_mean,
                   'MAE_std':ten_fold_BAG_MAE_std,
                   'MSE_mean': ten_fold_BAG_MSE_mean,
                   'MSE_std':ten_fold_BAG_MSE_std}
                  )
sbp_10_fold_BAG.style.set_caption("sbp - 10 Fold Cross Validation - BAGGING")


In [ ]:
BV_mean_MAE_mean = []
BV_mean_MAE_std = []
BV_mean_MSE_mean = []
BV_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_5_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_10_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_20_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_30_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_40_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_50_perc, 'sbp_final', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

sbp_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_mean_MAE_mean,
                   'MAE_std':BV_mean_MAE_std,
                   'MSE_mean': BV_mean_MSE_mean,
                   'MSE_std':BV_mean_MSE_std}
                  )
sbp_bv_mean.style.set_caption("sbp - Bootstrap Validation - MEAN")

In [ ]:
# k fold bootstrap method
BV_O_mean_MAE_mean = []
BV_O_mean_MAE_std = []
BV_O_mean_MSE_mean = []
BV_O_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_5_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_10_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_20_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_30_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_40_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_50_perc, 'sbp_final', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

sbp_bv_o_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_mean_MAE_mean,
                   'MAE_std':BV_O_mean_MAE_std,
                   'MSE_mean': BV_O_mean_MSE_mean,
                   'MSE_std':BV_O_mean_MSE_std}
                  )
sbp_bv_o_mean.style.set_caption("SBP - Bootstrap Validation (Original) - MEAN")

In [ ]:
# k fold bootstrap method
BV_5_fold_mean_MAE_mean = []
BV_5_fold_mean_MAE_std = []
BV_5_fold_mean_MSE_mean = []
BV_5_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_5_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_10_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_20_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_30_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_40_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_50_perc, 'sbp_final', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

sbp_5_fold_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_mean_MAE_mean,
                   'MAE_std':BV_5_fold_mean_MAE_std,
                   'MSE_mean': BV_5_fold_mean_MSE_mean,
                   'MSE_std':BV_5_fold_mean_MSE_std}
                  )
sbp_5_fold_bv_mean.style.set_caption("SBP - Bootstrap Validation K fold - MEAN")

In [ ]:
BV_median_MAE_mean = []
BV_median_MAE_std = []
BV_median_MSE_mean = []
BV_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_5_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_10_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_20_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_30_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_40_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_50_perc, 'sbp_final', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

sbp_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_median_MAE_mean,
                   'MAE_std':BV_median_MAE_std,
                   'MSE_mean': BV_median_MSE_mean,
                   'MSE_std':BV_median_MSE_std}
                  )
sbp_bv_median.style.set_caption("sbp - Bootstrap Validation - MEDIAN")

In [ ]:
BV_O_median_MAE_mean = []
BV_O_median_MAE_std = []
BV_O_median_MSE_mean = []
BV_O_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_5_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_10_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_20_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_30_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_40_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_bv_o_50_perc, 'sbp_final', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

sbp_bv_o_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_median_MAE_mean,
                   'MAE_std':BV_O_median_MAE_std,
                   'MSE_mean': BV_O_median_MSE_mean,
                   'MSE_std':BV_O_median_MSE_std}
                  )
sbp_bv_o_median.style.set_caption("SBP - Bootstrap Validation (Original) - MEDIAN")

In [ ]:
BV_5_fold_median_MAE_mean = []
BV_5_fold_median_MAE_std = []
BV_5_fold_median_MSE_mean = []
BV_5_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_5_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_10_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_20_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_30_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_40_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(sbp_5_fold_bv_50_perc, 'sbp_final', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

sbp_5_fold_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_median_MAE_mean,
                   'MAE_std':BV_5_fold_median_MAE_std,
                   'MSE_mean': BV_5_fold_median_MSE_mean,
                   'MSE_std':BV_5_fold_median_MSE_std}
                  )
sbp_5_fold_bv_median.style.set_caption("SBP - Bootstrap Validation K fold - MEDIAN")

In [ ]:
BV_Lin_MAE_mean = []
BV_Lin_MAE_std = []
BV_Lin_MSE_mean = []
BV_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_5_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_10_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_20_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_30_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_40_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_50_perc, linear_model, 'sbp_final')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

sbp_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_Lin_MAE_mean,
                   'MAE_std':BV_Lin_MAE_std,
                   'MSE_mean': BV_Lin_MSE_mean,
                   'MSE_std':BV_Lin_MSE_std}
                  )
sbp_bv_linear.style.set_caption("sbp - Bootstrap Validation - LINEAR REGRESSION")

In [ ]:
BV_O_Lin_MAE_mean = []
BV_O_Lin_MAE_std = []
BV_O_Lin_MSE_mean = []
BV_O_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_5_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_10_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_20_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_30_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_40_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_50_perc, linear_model, 'sbp_final')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

sbp_bv_o_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_Lin_MAE_mean,
                   'MAE_std':BV_O_Lin_MAE_std,
                   'MSE_mean': BV_O_Lin_MSE_mean,
                   'MSE_std':BV_O_Lin_MSE_std}
                  )
sbp_bv_o_linear.style.set_caption("SBP - Bootstrap Validation (Original) - LINEAR REGRESSION")

In [ ]:
BV_5_fold_Lin_MAE_mean = []
BV_5_fold_Lin_MAE_std = []
BV_5_fold_Lin_MSE_mean = []
BV_5_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_5_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_10_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_20_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_30_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_40_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_50_perc, linear_model, 'sbp_final')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

sbp_5_fold_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_Lin_MAE_mean,
                   'MAE_std':BV_5_fold_Lin_MAE_std,
                   'MSE_mean': BV_5_fold_Lin_MSE_mean,
                   'MSE_std':BV_5_fold_Lin_MSE_std}
                  )
sbp_5_fold_bv_linear.style.set_caption("SBP - Bootstrap Validation K fold - LINEAR REGRESSION")

In [ ]:
BV_RF_MAE_mean = []
BV_RF_MAE_std = []
BV_RF_MSE_mean = []
BV_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_5_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_10_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_20_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_30_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_40_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_50_perc, random_forest_model, 'sbp_final')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

sbp_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_RF_MAE_mean,
                   'MAE_std':BV_RF_MAE_std,
                   'MSE_mean': BV_RF_MSE_mean,
                   'MSE_std':BV_RF_MSE_std}
                  )
sbp_bv_random_forest.style.set_caption("sbp - Bootstrap Validation - RANDOM FOREST REGRESSION")

In [ ]:
BV_O_RF_MAE_mean = []
BV_O_RF_MAE_std = []
BV_O_RF_MSE_mean = []
BV_O_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_5_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_10_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_20_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_30_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_40_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_50_perc, random_forest_model, 'sbp_final')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

sbp_bv_o_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_RF_MAE_mean,
                   'MAE_std':BV_O_RF_MAE_std,
                   'MSE_mean': BV_O_RF_MSE_mean,
                   'MSE_std':BV_O_RF_MSE_std}
                  )
sbp_bv_o_random_forest.style.set_caption("SBP - Bootstrap Validation (Original) - RANDOM FOREST REGRESSION")

In [ ]:
BV_5_fold_RF_MAE_mean = []
BV_5_fold_RF_MAE_std = []
BV_5_fold_RF_MSE_mean = []
BV_5_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_5_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_10_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_20_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_30_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_40_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_50_perc, random_forest_model, 'sbp_final')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

sbp_5_fold_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_RF_MAE_mean,
                   'MAE_std':BV_5_fold_RF_MAE_std,
                   'MSE_mean': BV_5_fold_RF_MSE_mean,
                   'MSE_std':BV_5_fold_RF_MSE_std}
                  )
sbp_5_fold_bv_random_forest.style.set_caption("SBP - Bootstrap Validation K fold - RANDOM FOREST REGRESSION")

In [ ]:
BV_XGB_MAE_mean = []
BV_XGB_MAE_std = []
BV_XGB_MSE_mean = []
BV_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_5_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_10_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_20_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_30_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_40_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_50_perc, xgboost_model, 'sbp_final')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

sbp_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_XGB_MAE_mean,
                   'MAE_std':BV_XGB_MAE_std,
                   'MSE_mean': BV_XGB_MSE_mean,
                   'MSE_std':BV_XGB_MSE_std}
                  )
sbp_bv_XGB.style.set_caption("sbp - Bootstrap Validation - BOOSTING")

In [ ]:
BV_O_XGB_MAE_mean = []
BV_O_XGB_MAE_std = []
BV_O_XGB_MSE_mean = []
BV_O_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_5_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_10_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_20_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_30_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_40_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_50_perc, xgboost_model, 'sbp_final')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

sbp_bv_o_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_XGB_MAE_mean,
                   'MAE_std':BV_O_XGB_MAE_std,
                   'MSE_mean': BV_O_XGB_MSE_mean,
                   'MSE_std':BV_O_XGB_MSE_std}
                  )
sbp_bv_o_XGB.style.set_caption("SBP - Bootstrap Validation (Original) - BOOSTING")

In [ ]:
BV_5_fold_XGB_MAE_mean = []
BV_5_fold_XGB_MAE_std = []
BV_5_fold_XGB_MSE_mean = []
BV_5_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_5_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_10_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_20_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_30_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_40_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_50_perc, xgboost_model, 'sbp_final')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

sbp_5_fold_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_XGB_MAE_mean,
                   'MAE_std':BV_5_fold_XGB_MAE_std,
                   'MSE_mean': BV_5_fold_XGB_MSE_mean,
                   'MSE_std':BV_5_fold_XGB_MSE_std}
                  )
sbp_5_fold_bv_XGB.style.set_caption("SBP - Bootstrap Validation K fold - BOOSTING")

In [ ]:
sbp_bv_XGB_temp = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': [3.252333, 3.404642, 3.452825, 3.537449, 3.546953, 3.668054],
                   }
                  )
sbp_bv_XGB_temp.style.set_caption("sbp - Bootstrap Validation - BOOSTING")



In [ ]:
BV_BAG_MAE_mean = []
BV_BAG_MAE_std = []
BV_BAG_MSE_mean = []
BV_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_5_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_10_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_20_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_30_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_40_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_50_perc, bagging_model, 'sbp_final')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)

sbp_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_BAG_MAE_mean,
                   'MAE_std':BV_BAG_MAE_std,
                   'MSE_mean': BV_BAG_MSE_mean,
                   'MSE_std':BV_BAG_MSE_std}
                  )
sbp_bv_BAG.style.set_caption("sbp - Bootstrap Validation - BAGGING")

In [ ]:
BV_O_BAG_MAE_mean = []
BV_O_BAG_MAE_std = []
BV_O_BAG_MSE_mean = []
BV_O_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_5_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_10_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_20_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_30_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_40_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_bv_o_50_perc, bagging_model, 'sbp_final')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)

sbp_bv_o_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_BAG_MAE_mean,
                   'MAE_std':BV_O_BAG_MAE_std,
                   'MSE_mean': BV_O_BAG_MSE_mean,
                   'MSE_std':BV_O_BAG_MSE_std}
                  )
sbp_bv_o_BAG.style.set_caption("SBP - Bootstrap Validation (Original) - BAGGING")

In [ ]:
BV_5_fold_BAG_MAE_mean = []
BV_5_fold_BAG_MAE_std = []
BV_5_fold_BAG_MSE_mean = []
BV_5_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_5_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_10_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_20_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_30_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_40_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(sbp_5_fold_bv_50_perc, bagging_model, 'sbp_final')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)

sbp_5_fold_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_BAG_MAE_mean,
                   'MAE_std':BV_5_fold_BAG_MAE_std,
                   'MSE_mean': BV_5_fold_BAG_MSE_mean,
                   'MSE_std':BV_5_fold_BAG_MSE_std}
                  )
sbp_5_fold_bv_BAG.style.set_caption("SBP - Bootstrap Validation K fold - BAGGING")

# Respiratory Rate

In [ ]:
five_fold_mean_MAE_mean = []
five_fold_mean_MAE_std = []
five_fold_mean_MSE_mean = []
five_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_5_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_10_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_20_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_30_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_40_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_50_perc, 'Respiratory_Rate_br_per_min', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

resp_5_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_mean_MAE_mean,
                   'MAE_std':five_fold_mean_MAE_std,
                   'MSE_mean': five_fold_mean_MSE_mean,
                   'MSE_std':five_fold_mean_MSE_std}
                  )
resp_5_fold_mean.style.set_caption("resp - 5 Fold Cross Validation - MEAN")


In [ ]:

five_fold_median_MAE_mean = []
five_fold_median_MAE_std = []
five_fold_median_MSE_mean = []
five_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_5_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_10_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_20_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_30_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_40_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_folds_50_perc, 'Respiratory_Rate_br_per_min', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

resp_5_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_median_MAE_mean,
                   'MAE_std':five_fold_median_MAE_std,
                   'MSE_mean': five_fold_median_MSE_mean,
                   'MSE_std':five_fold_median_MSE_std}
                  )
resp_5_fold_median.style.set_caption("resp - 5 Fold Cross Validation - MEDIAN")


In [ ]:
five_fold_Lin_MAE_mean = []
five_fold_Lin_MAE_std = []
five_fold_Lin_MSE_mean = []
five_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_5_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_10_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_20_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_30_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_40_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_50_perc, linear_model, 'Respiratory_Rate_br_per_min')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

resp_5_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_Lin_MAE_mean,
                   'MAE_std':five_fold_Lin_MAE_std,
                   'MSE_mean': five_fold_Lin_MSE_mean,
                   'MSE_std':five_fold_Lin_MSE_std}
                  )
resp_5_fold_linear.style.set_caption("Respiratory - 5 Fold Cross Validation - LINEAR REGRESSION")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

five_fold_RF_MAE_mean = []
five_fold_RF_MAE_std = []
five_fold_RF_MSE_mean = []
five_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_5_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_10_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_20_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_30_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_40_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_50_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

resp_5_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_RF_MAE_mean,
                   'MAE_std':five_fold_RF_MAE_std,
                   'MSE_mean': five_fold_RF_MSE_mean,
                   'MSE_std':five_fold_RF_MSE_std}
                  )
resp_5_fold_random_forest.style.set_caption("resp - 5 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
five_fold_XGB_MAE_mean = []
five_fold_XGB_MAE_std = []
five_fold_XGB_MSE_mean = []
five_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_5_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_10_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_20_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_30_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_40_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_50_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

resp_5_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_XGB_MAE_mean,
                   'MAE_std':five_fold_XGB_MAE_std,
                   'MSE_mean': five_fold_XGB_MSE_mean,
                   'MSE_std':five_fold_XGB_MSE_std}
                  )
resp_5_fold_XGB.style.set_caption("resp - 5 Fold Cross Validation - BOOSTING")


In [ ]:
five_fold_BAG_MAE_mean = []
five_fold_BAG_MAE_std = []
five_fold_BAG_MSE_mean = []
five_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_5_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_10_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_20_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_30_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_40_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_folds_50_perc, bagging_model, 'Respiratory_Rate_br_per_min')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
resp_5_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_BAG_MAE_mean,
                   'MAE_std':five_fold_BAG_MAE_std,
                   'MSE_mean': five_fold_BAG_MSE_mean,
                   'MSE_std':five_fold_BAG_MSE_std}
                  )
resp_5_fold_BAG.style.set_caption("resp - 5 Fold Cross Validation - BAGGING")


In [ ]:
ten_fold_mean_MAE_mean = []
ten_fold_mean_MAE_std = []
ten_fold_mean_MSE_mean = []
ten_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_5_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_10_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_20_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_30_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_40_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_50_perc, 'Respiratory_Rate_br_per_min', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

resp_10_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_mean_MAE_mean,
                   'MAE_std':ten_fold_mean_MAE_std,
                   'MSE_mean': ten_fold_mean_MSE_mean,
                   'MSE_std':ten_fold_mean_MSE_std}
                  )
resp_10_fold_mean.style.set_caption("resp - 10 Fold Cross Validation - MEAN")


In [ ]:
ten_fold_median_MAE_mean = []
ten_fold_median_MAE_std = []
ten_fold_median_MSE_mean = []
ten_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_5_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_10_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_20_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_30_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_40_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_10_folds_50_perc, 'Respiratory_Rate_br_per_min', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

resp_10_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_median_MAE_mean,
                   'MAE_std':ten_fold_median_MAE_std,
                   'MSE_mean': ten_fold_median_MSE_mean,
                   'MSE_std':ten_fold_median_MSE_std}
                  )
resp_10_fold_median.style.set_caption("resp - 10 Fold Cross Validation - MEDIAN")


In [ ]:
ten_fold_Lin_MAE_mean = []
ten_fold_Lin_MAE_std = []
ten_fold_Lin_MSE_mean = []
ten_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_5_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_10_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_20_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_30_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_40_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_50_perc, linear_model, 'Respiratory_Rate_br_per_min')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

resp_10_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_Lin_MAE_mean,
                   'MAE_std':ten_fold_Lin_MAE_std,
                   'MSE_mean': ten_fold_Lin_MSE_mean,
                   'MSE_std':ten_fold_Lin_MSE_std}
                  )
resp_10_fold_linear.style.set_caption("resp - 10 Fold Cross Validation - LINEAR REGRESSION")

In [ ]:
ten_fold_RF_MAE_mean = []
ten_fold_RF_MAE_std = []
ten_fold_RF_MSE_mean = []
ten_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_5_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_10_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_20_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_30_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_40_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_50_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

resp_10_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_RF_MAE_mean,
                   'MAE_std':ten_fold_RF_MAE_std,
                   'MSE_mean': ten_fold_RF_MSE_mean,
                   'MSE_std':ten_fold_RF_MSE_std}
                  )
resp_10_fold_random_forest.style.set_caption("resp - 10 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
ten_fold_XGB_MAE_mean = []
ten_fold_XGB_MAE_std = []
ten_fold_XGB_MSE_mean = []
ten_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_5_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_10_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_20_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_30_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_40_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_50_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

resp_10_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_XGB_MAE_mean,
                   'MAE_std':ten_fold_XGB_MAE_std,
                   'MSE_mean': ten_fold_XGB_MSE_mean,
                   'MSE_std':ten_fold_XGB_MSE_std}
                  )
resp_10_fold_XGB.style.set_caption("resp - 10 Fold Cross Validation - BOOSTING")


In [ ]:
ten_fold_BAG_MAE_mean = []
ten_fold_BAG_MAE_std = []
ten_fold_BAG_MSE_mean = []
ten_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_5_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_10_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_20_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_30_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_40_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_10_folds_50_perc, bagging_model, 'Respiratory_Rate_br_per_min')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)

resp_10_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_BAG_MAE_mean,
                   'MAE_std':ten_fold_BAG_MAE_std,
                   'MSE_mean': ten_fold_BAG_MSE_mean,
                   'MSE_std':ten_fold_BAG_MSE_std}
                  )
resp_10_fold_BAG.style.set_caption("resp - 10 Fold Cross Validation - BAGGING")

In [ ]:
BV_mean_MAE_mean = []
BV_mean_MAE_std = []
BV_mean_MSE_mean = []
BV_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_5_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_10_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_20_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_30_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_40_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_50_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

resp_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_mean_MAE_mean,
                   'MAE_std':BV_mean_MAE_std,
                   'MSE_mean': BV_mean_MSE_mean,
                   'MSE_std':BV_mean_MSE_std}
                  )
resp_bv_mean.style.set_caption("resp - Bootstrap Validation - MEAN")

In [ ]:
# Original Bootstrap method
BV_O_mean_MAE_mean = []
BV_O_mean_MAE_std = []
BV_O_mean_MSE_mean = []
BV_O_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_5_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_10_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_20_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_30_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_40_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_50_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

resp_bv_o_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_mean_MAE_mean,
                   'MAE_std':BV_O_mean_MAE_std,
                   'MSE_mean': BV_O_mean_MSE_mean,
                   'MSE_std':BV_O_mean_MSE_std}
                  )
resp_bv_o_mean.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - MEAN")

In [ ]:
# k fold bootstrap method
BV_5_fold_mean_MAE_mean = []
BV_5_fold_mean_MAE_std = []
BV_5_fold_mean_MSE_mean = []
BV_5_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_5_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_10_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_20_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_30_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_40_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_50_perc, 'Respiratory_Rate_br_per_min', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

resp_5_fold_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_mean_MAE_mean,
                   'MAE_std':BV_5_fold_mean_MAE_std,
                   'MSE_mean': BV_5_fold_mean_MSE_mean,
                   'MSE_std':BV_5_fold_mean_MSE_std}
                  )
resp_5_fold_bv_mean.style.set_caption("Respiratory rate - Bootstrap Validation 5 fold - MEAN")

In [ ]:
BV_median_MAE_mean = []
BV_median_MAE_std = []
BV_median_MSE_mean = []
BV_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_5_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_10_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_20_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_30_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_40_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_50_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

resp_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_median_MAE_mean,
                   'MAE_std':BV_median_MAE_std,
                   'MSE_mean': BV_median_MSE_mean,
                   'MSE_std':BV_median_MSE_std}
                  )
resp_bv_median.style.set_caption("resp - Bootstrap Validation - MEDIAN")


In [ ]:
BV_O_median_MAE_mean = []
BV_O_median_MAE_std = []
BV_O_median_MSE_mean = []
BV_O_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_5_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_10_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_20_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_30_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_40_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_bv_o_50_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

resp_bv_o_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_median_MAE_mean,
                   'MAE_std':BV_O_median_MAE_std,
                   'MSE_mean': BV_O_median_MSE_mean,
                   'MSE_std':BV_O_median_MSE_std}
                  )
resp_bv_o_median.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - MEDIAN")

In [ ]:
BV_5_fold_median_MAE_mean = []
BV_5_fold_median_MAE_std = []
BV_5_fold_median_MSE_mean = []
BV_5_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_5_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_10_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_20_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_30_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_40_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(resp_5_fold_bv_50_perc, 'Respiratory_Rate_br_per_min', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

resp_5_fold_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_median_MAE_mean,
                   'MAE_std':BV_5_fold_median_MAE_std,
                   'MSE_mean': BV_5_fold_median_MSE_mean,
                   'MSE_std':BV_5_fold_median_MSE_std}
                  )
resp_5_fold_bv_median.style.set_caption("Respiratory Rate - Bootstrap Validation 5 fold - MEDIAN")

In [ ]:
BV_Lin_MAE_mean = []
BV_Lin_MAE_std = []
BV_Lin_MSE_mean = []
BV_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_5_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_10_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_20_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_30_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_40_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_50_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

resp_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_Lin_MAE_mean,
                   'MAE_std':BV_Lin_MAE_std,
                   'MSE_mean': BV_Lin_MSE_mean,
                   'MSE_std':BV_Lin_MSE_std}
                  )
resp_bv_linear.style.set_caption("resp - Bootstrap Validation - LINEAR REGRESSION")


In [ ]:
BV_O_Lin_MAE_mean = []
BV_O_Lin_MAE_std = []
BV_O_Lin_MSE_mean = []
BV_O_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_5_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_10_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_20_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_30_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_40_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_50_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

resp_bv_o_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_Lin_MAE_mean,
                   'MAE_std':BV_O_Lin_MAE_std,
                   'MSE_mean': BV_O_Lin_MSE_mean,
                   'MSE_std':BV_O_Lin_MSE_std}
                  )
resp_bv_o_linear.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - LINEAR REGRESSION")

In [ ]:
BV_5_fold_Lin_MAE_mean = []
BV_5_fold_Lin_MAE_std = []
BV_5_fold_Lin_MSE_mean = []
BV_5_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_5_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_10_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_20_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_30_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_40_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_50_perc, linear_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

resp_5_fold_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_Lin_MAE_mean,
                   'MAE_std':BV_5_fold_Lin_MAE_std,
                   'MSE_mean': BV_5_fold_Lin_MSE_mean,
                   'MSE_std':BV_5_fold_Lin_MSE_std}
                  )
resp_5_fold_bv_linear.style.set_caption("Respiratory Rate - Bootstrap Validation 5 fold - LINEAR REGRESSION")

In [ ]:
BV_RF_MAE_mean = []
BV_RF_MAE_std = []
BV_RF_MSE_mean = []
BV_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_5_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_10_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_20_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_30_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_40_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_50_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

resp_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_RF_MAE_mean,
                   'MAE_std':BV_RF_MAE_std,
                   'MSE_mean': BV_RF_MSE_mean,
                   'MSE_std':BV_RF_MSE_std}
                  )
resp_bv_random_forest.style.set_caption("resp - Bootstrap Validation - RANDOM FOREST REGRESSION")


In [ ]:
BV_O_RF_MAE_mean = []
BV_O_RF_MAE_std = []
BV_O_RF_MSE_mean = []
BV_O_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_5_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_10_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_20_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_30_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_40_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_50_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

resp_bv_o_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_RF_MAE_mean,
                   'MAE_std':BV_O_RF_MAE_std,
                   'MSE_mean': BV_O_RF_MSE_mean,
                   'MSE_std':BV_O_RF_MSE_std}
                  )
resp_bv_o_random_forest.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - RANDOM FOREST REGRESSION")

In [ ]:
BV_5_fold_RF_MAE_mean = []
BV_5_fold_RF_MAE_std = []
BV_5_fold_RF_MSE_mean = []
BV_5_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_5_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_10_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_20_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_30_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_40_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_50_perc, random_forest_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

resp_5_fold_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_RF_MAE_mean,
                   'MAE_std':BV_5_fold_RF_MAE_std,
                   'MSE_mean': BV_5_fold_RF_MSE_mean,
                   'MSE_std':BV_5_fold_RF_MSE_std}
                  )
resp_5_fold_bv_random_forest.style.set_caption("Respiratory Rate - Bootstrap Validation 5 fold - RANDOM FOREST REGRESSION")

In [ ]:
BV_XGB_MAE_mean = []
BV_XGB_MAE_std = []
BV_XGB_MSE_mean = []
BV_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_5_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_10_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_20_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_30_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_40_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_50_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

resp_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_XGB_MAE_mean,
                   'MAE_std':BV_XGB_MAE_std,
                   'MSE_mean': BV_XGB_MSE_mean,
                   'MSE_std':BV_XGB_MSE_std}
                  )
resp_bv_XGB.style.set_caption("resp - Bootstrap Validation - BOOSTING")


In [ ]:
BV_O_XGB_MAE_mean = []
BV_O_XGB_MAE_std = []
BV_O_XGB_MSE_mean = []
BV_O_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_5_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_10_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_20_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_30_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_40_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_50_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

resp_bv_o_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_XGB_MAE_mean,
                   'MAE_std':BV_O_XGB_MAE_std,
                   'MSE_mean': BV_O_XGB_MSE_mean,
                   'MSE_std':BV_O_XGB_MSE_std}
                  )
resp_bv_o_XGB.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - BOOSTING")

In [ ]:
BV_5_fold_XGB_MAE_mean = []
BV_5_fold_XGB_MAE_std = []
BV_5_fold_XGB_MSE_mean = []
BV_5_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_5_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_10_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_20_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_30_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_40_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_50_perc, xgboost_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

resp_5_fold_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_XGB_MAE_mean,
                   'MAE_std':BV_5_fold_XGB_MAE_std,
                   'MSE_mean': BV_5_fold_XGB_MSE_mean,
                   'MSE_std':BV_5_fold_XGB_MSE_std}
                  )
resp_5_fold_bv_XGB.style.set_caption("Respiratory Rate - Bootstrap Validation 5 fold - BOOSTING")

In [ ]:
BV_BAG_MAE_mean = []
BV_BAG_MAE_std = []
BV_BAG_MSE_mean = []
BV_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_5_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_10_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_20_perc, bagging_model, 'Respiratory_Rate_br_per_min') 
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_30_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_40_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_50_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)

resp_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_BAG_MAE_mean,
                   'MAE_std':BV_BAG_MAE_std,
                   'MSE_mean': BV_BAG_MSE_mean,
                   'MSE_std':BV_BAG_MSE_std}
                  )
resp_bv_BAG.style.set_caption("resp - Bootstrap Validation - BAGGING")


In [ ]:
BV_O_BAG_MAE_mean = []
BV_O_BAG_MAE_std = []
BV_O_BAG_MSE_mean = []
BV_O_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_5_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_10_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_20_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_30_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_40_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_bv_o_50_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)

resp_bv_o_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_BAG_MAE_mean,
                   'MAE_std':BV_O_BAG_MAE_std,
                   'MSE_mean': BV_O_BAG_MSE_mean,
                   'MSE_std':BV_O_BAG_MSE_std}
                  )
resp_bv_o_BAG.style.set_caption("Respiratory Rate - Bootstrap Validation (Original) - BAGGING")

In [ ]:
BV_5_fold_BAG_MAE_mean = []
BV_5_fold_BAG_MAE_std = []
BV_5_fold_BAG_MSE_mean = []
BV_5_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_5_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_10_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_20_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_30_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_40_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(resp_5_fold_bv_50_perc, bagging_model, 'Respiratory_Rate_br_per_min')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)

resp_5_fold_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_BAG_MAE_mean,
                   'MAE_std':BV_5_fold_BAG_MAE_std,
                   'MSE_mean': BV_5_fold_BAG_MSE_mean,
                   'MSE_std':BV_5_fold_BAG_MSE_std}
                  )
resp_5_fold_bv_BAG.style.set_caption("Respiratory Rate - Bootstrap Validation 5 fold - BAGGING")

# Pulse Rate

In [ ]:
five_fold_mean_MAE_mean = []
five_fold_mean_MAE_std = []
five_fold_mean_MSE_mean = []
five_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_5_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_10_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_20_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_30_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_40_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_50_perc, 'peripheral_pulse_rate', 'mean')
five_fold_mean_MAE_mean.append(mae_m)
five_fold_mean_MAE_std.append(mae_s)
five_fold_mean_MSE_mean.append(mse_m)
five_fold_mean_MSE_std.append(mse_s)

pulse_5_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_mean_MAE_mean,
                   'MAE_std':five_fold_mean_MAE_std,
                   'MSE_mean': five_fold_mean_MSE_mean,
                   'MSE_std':five_fold_mean_MSE_std}
                  )
pulse_5_fold_mean.style.set_caption("pulse - 5 Fold Cross Validation - MEAN")


In [ ]:

five_fold_median_MAE_mean = []
five_fold_median_MAE_std = []
five_fold_median_MSE_mean = []
five_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_5_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_10_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_20_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_30_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_40_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_folds_50_perc, 'peripheral_pulse_rate', 'median')
five_fold_median_MAE_mean.append(mae_m)
five_fold_median_MAE_std.append(mae_s)
five_fold_median_MSE_mean.append(mse_m)
five_fold_median_MSE_std.append(mse_s)

pulse_5_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_median_MAE_mean,
                   'MAE_std':five_fold_median_MAE_std,
                   'MSE_mean': five_fold_median_MSE_mean,
                   'MSE_std':five_fold_median_MSE_std}
                  )
pulse_5_fold_median.style.set_caption("pulse - 5 Fold Cross Validation - MEDIAN")


In [ ]:
five_fold_Lin_MAE_mean = []
five_fold_Lin_MAE_std = []
five_fold_Lin_MSE_mean = []
five_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_5_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_10_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_20_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_30_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_40_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_50_perc, linear_model, 'peripheral_pulse_rate')
five_fold_Lin_MAE_mean.append(mae_m)
five_fold_Lin_MAE_std.append(mae_s)
five_fold_Lin_MSE_mean.append(mse_m)
five_fold_Lin_MSE_std.append(mse_s)

pulse_5_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_Lin_MAE_mean,
                   'MAE_std':five_fold_Lin_MAE_std,
                   'MSE_mean': five_fold_Lin_MSE_mean,
                   'MSE_std':five_fold_Lin_MSE_std}
                  )
pulse_5_fold_linear.style.set_caption("pulseiratory - 5 Fold Cross Validation - LINEAR REGRESSION")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

five_fold_RF_MAE_mean = []
five_fold_RF_MAE_std = []
five_fold_RF_MSE_mean = []
five_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_5_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_10_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_20_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_30_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_40_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_50_perc, random_forest_model, 'peripheral_pulse_rate')
five_fold_RF_MAE_mean.append(mae_m)
five_fold_RF_MAE_std.append(mae_s)
five_fold_RF_MSE_mean.append(mse_m)
five_fold_RF_MSE_std.append(mse_s)

pulse_5_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_RF_MAE_mean,
                   'MAE_std':five_fold_RF_MAE_std,
                   'MSE_mean': five_fold_RF_MSE_mean,
                   'MSE_std':five_fold_RF_MSE_std}
                  )
pulse_5_fold_random_forest.style.set_caption("pulse - 5 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
five_fold_XGB_MAE_mean = []
five_fold_XGB_MAE_std = []
five_fold_XGB_MSE_mean = []
five_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_5_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_10_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_20_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_30_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_40_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_50_perc, xgboost_model, 'peripheral_pulse_rate')
five_fold_XGB_MAE_mean.append(mae_m)
five_fold_XGB_MAE_std.append(mae_s)
five_fold_XGB_MSE_mean.append(mse_m)
five_fold_XGB_MSE_std.append(mse_s)

pulse_5_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_XGB_MAE_mean,
                   'MAE_std':five_fold_XGB_MAE_std,
                   'MSE_mean': five_fold_XGB_MSE_mean,
                   'MSE_std':five_fold_XGB_MSE_std}
                  )
pulse_5_fold_XGB.style.set_caption("pulse - 5 Fold Cross Validation - BOOSTING")


In [ ]:
five_fold_BAG_MAE_mean = []
five_fold_BAG_MAE_std = []
five_fold_BAG_MSE_mean = []
five_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_5_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_10_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_20_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_30_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_40_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_folds_50_perc, bagging_model, 'peripheral_pulse_rate')
five_fold_BAG_MAE_mean.append(mae_m)
five_fold_BAG_MAE_std.append(mae_s)
five_fold_BAG_MSE_mean.append(mse_m)
five_fold_BAG_MSE_std.append(mse_s)
pulse_5_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': five_fold_BAG_MAE_mean,
                   'MAE_std':five_fold_BAG_MAE_std,
                   'MSE_mean': five_fold_BAG_MSE_mean,
                   'MSE_std':five_fold_BAG_MSE_std}
                  )
pulse_5_fold_BAG.style.set_caption("pulse - 5 Fold Cross Validation - BAGGING")


In [ ]:
ten_fold_mean_MAE_mean = []
ten_fold_mean_MAE_std = []
ten_fold_mean_MSE_mean = []
ten_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_5_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_10_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_20_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_30_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_40_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_50_perc, 'peripheral_pulse_rate', 'mean')
ten_fold_mean_MAE_mean.append(mae_m)
ten_fold_mean_MAE_std.append(mae_s)
ten_fold_mean_MSE_mean.append(mse_m)
ten_fold_mean_MSE_std.append(mse_s)

pulse_10_fold_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_mean_MAE_mean,
                   'MAE_std':ten_fold_mean_MAE_std,
                   'MSE_mean': ten_fold_mean_MSE_mean,
                   'MSE_std':ten_fold_mean_MSE_std}
                  )
pulse_10_fold_mean.style.set_caption("pulse - 10 Fold Cross Validation - MEAN")


In [ ]:
ten_fold_median_MAE_mean = []
ten_fold_median_MAE_std = []
ten_fold_median_MSE_mean = []
ten_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_5_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_10_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_20_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_30_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_40_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_10_folds_50_perc, 'peripheral_pulse_rate', 'median')
ten_fold_median_MAE_mean.append(mae_m)
ten_fold_median_MAE_std.append(mae_s)
ten_fold_median_MSE_mean.append(mse_m)
ten_fold_median_MSE_std.append(mse_s)

pulse_10_fold_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_median_MAE_mean,
                   'MAE_std':ten_fold_median_MAE_std,
                   'MSE_mean': ten_fold_median_MSE_mean,
                   'MSE_std':ten_fold_median_MSE_std}
                  )
pulse_10_fold_median.style.set_caption("pulse - 10 Fold Cross Validation - MEDIAN")


In [ ]:
ten_fold_Lin_MAE_mean = []
ten_fold_Lin_MAE_std = []
ten_fold_Lin_MSE_mean = []
ten_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_5_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_10_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_20_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_30_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_40_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_50_perc, linear_model, 'peripheral_pulse_rate')
ten_fold_Lin_MAE_mean.append(mae_m)
ten_fold_Lin_MAE_std.append(mae_s)
ten_fold_Lin_MSE_mean.append(mse_m)
ten_fold_Lin_MSE_std.append(mse_s)

pulse_10_fold_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_Lin_MAE_mean,
                   'MAE_std':ten_fold_Lin_MAE_std,
                   'MSE_mean': ten_fold_Lin_MSE_mean,
                   'MSE_std':ten_fold_Lin_MSE_std}
                  )
pulse_10_fold_linear.style.set_caption("pulse - 10 Fold Cross Validation - LINEAR REGRESSION")


In [ ]:
ten_fold_RF_MAE_mean = []
ten_fold_RF_MAE_std = []
ten_fold_RF_MSE_mean = []
ten_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_5_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_10_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_20_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_30_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_40_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_50_perc, random_forest_model, 'peripheral_pulse_rate')
ten_fold_RF_MAE_mean.append(mae_m)
ten_fold_RF_MAE_std.append(mae_s)
ten_fold_RF_MSE_mean.append(mse_m)
ten_fold_RF_MSE_std.append(mse_s)

pulse_10_fold_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_RF_MAE_mean,
                   'MAE_std':ten_fold_RF_MAE_std,
                   'MSE_mean': ten_fold_RF_MSE_mean,
                   'MSE_std':ten_fold_RF_MSE_std}
                  )
pulse_10_fold_random_forest.style.set_caption("pulse - 10 Fold Cross Validation - RANDOM FOREST REGRESSION")


In [ ]:
ten_fold_XGB_MAE_mean = []
ten_fold_XGB_MAE_std = []
ten_fold_XGB_MSE_mean = []
ten_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_5_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_10_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_20_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_30_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_40_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_50_perc, xgboost_model, 'peripheral_pulse_rate')
ten_fold_XGB_MAE_mean.append(mae_m)
ten_fold_XGB_MAE_std.append(mae_s)
ten_fold_XGB_MSE_mean.append(mse_m)
ten_fold_XGB_MSE_std.append(mse_s)

pulse_10_fold_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_XGB_MAE_mean,
                   'MAE_std':ten_fold_XGB_MAE_std,
                   'MSE_mean': ten_fold_XGB_MSE_mean,
                   'MSE_std':ten_fold_XGB_MSE_std}
                  )
pulse_10_fold_XGB.style.set_caption("pulse - 10 Fold Cross Validation - BOOSTING")


In [ ]:
ten_fold_BAG_MAE_mean = []
ten_fold_BAG_MAE_std = []
ten_fold_BAG_MSE_mean = []
ten_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_5_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_10_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_20_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_30_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_40_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_10_folds_50_perc, bagging_model, 'peripheral_pulse_rate')
ten_fold_BAG_MAE_mean.append(mae_m)
ten_fold_BAG_MAE_std.append(mae_s)
ten_fold_BAG_MSE_mean.append(mse_m)
ten_fold_BAG_MSE_std.append(mse_s)

pulse_10_fold_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': ten_fold_BAG_MAE_mean,
                   'MAE_std':ten_fold_BAG_MAE_std,
                   'MSE_mean': ten_fold_BAG_MSE_mean,
                   'MSE_std':ten_fold_BAG_MSE_std}
                  )
pulse_10_fold_BAG.style.set_caption("pulse - 10 Fold Cross Validation - BAGGING")


In [ ]:
BV_mean_MAE_mean = []
BV_mean_MAE_std = []
BV_mean_MSE_mean = []
BV_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_5_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_10_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_20_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_30_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_40_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_50_perc, 'peripheral_pulse_rate', 'mean')
BV_mean_MAE_mean.append(mae_m)
BV_mean_MAE_std.append(mae_s)
BV_mean_MSE_mean.append(mse_m)
BV_mean_MSE_std.append(mse_s)

pulse_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_mean_MAE_mean,
                   'MAE_std':BV_mean_MAE_std,
                   'MSE_mean': BV_mean_MSE_mean,
                   'MSE_std':BV_mean_MSE_std}
                  )
pulse_bv_mean.style.set_caption("pulse - Bootstrap Validation - MEAN")


In [ ]:
# k fold bootstrap method
BV_O_mean_MAE_mean = []
BV_O_mean_MAE_std = []
BV_O_mean_MSE_mean = []
BV_O_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_5_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_10_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_20_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_30_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_40_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_50_perc, 'peripheral_pulse_rate', 'mean')
BV_O_mean_MAE_mean.append(mae_m)
BV_O_mean_MAE_std.append(mae_s)
BV_O_mean_MSE_mean.append(mse_m)
BV_O_mean_MSE_std.append(mse_s)

pulse_bv_o_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_mean_MAE_mean,
                   'MAE_std':BV_O_mean_MAE_std,
                   'MSE_mean': BV_O_mean_MSE_mean,
                   'MSE_std':BV_O_mean_MSE_std}
                  )
pulse_bv_o_mean.style.set_caption("Pulse - Bootstrap Validation (Original) - MEAN")

In [ ]:
# k fold bootstrap method
BV_5_fold_mean_MAE_mean = []
BV_5_fold_mean_MAE_std = []
BV_5_fold_mean_MSE_mean = []
BV_5_fold_mean_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_5_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_10_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_20_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_30_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_40_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_50_perc, 'peripheral_pulse_rate', 'mean')
BV_5_fold_mean_MAE_mean.append(mae_m)
BV_5_fold_mean_MAE_std.append(mae_s)
BV_5_fold_mean_MSE_mean.append(mse_m)
BV_5_fold_mean_MSE_std.append(mse_s)

pulse_5_fold_bv_mean = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_mean_MAE_mean,
                   'MAE_std':BV_5_fold_mean_MAE_std,
                   'MSE_mean': BV_5_fold_mean_MSE_mean,
                   'MSE_std':BV_5_fold_mean_MSE_std}
                  )
pulse_5_fold_bv_mean.style.set_caption("Pulse - Bootstrap Validation 5 fold - MEAN")

In [ ]:
BV_median_MAE_mean = []
BV_median_MAE_std = []
BV_median_MSE_mean = []
BV_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_5_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_10_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_20_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_30_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_40_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_50_perc, 'peripheral_pulse_rate', 'median')
BV_median_MAE_mean.append(mae_m)
BV_median_MAE_std.append(mae_s)
BV_median_MSE_mean.append(mse_m)
BV_median_MSE_std.append(mse_s)

pulse_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_median_MAE_mean,
                   'MAE_std':BV_median_MAE_std,
                   'MSE_mean': BV_median_MSE_mean,
                   'MSE_std':BV_median_MSE_std}
                  )
pulse_bv_median.style.set_caption("pulse - Bootstrap Validation - MEDIAN")


In [ ]:
BV_O_median_MAE_mean = []
BV_O_median_MAE_std = []
BV_O_median_MSE_mean = []
BV_O_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_5_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_10_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_20_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_30_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_40_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_bv_o_50_perc, 'peripheral_pulse_rate', 'median')
BV_O_median_MAE_mean.append(mae_m)
BV_O_median_MAE_std.append(mae_s)
BV_O_median_MSE_mean.append(mse_m)
BV_O_median_MSE_std.append(mse_s)

pulse_bv_o_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_median_MAE_mean,
                   'MAE_std':BV_O_median_MAE_std,
                   'MSE_mean': BV_O_median_MSE_mean,
                   'MSE_std':BV_O_median_MSE_std}
                  )
pulse_bv_o_median.style.set_caption("Pulse - Bootstrap Validation (Original) - MEDIAN")

In [ ]:
BV_5_fold_median_MAE_mean = []
BV_5_fold_median_MAE_std = []
BV_5_fold_median_MSE_mean = []
BV_5_fold_median_MSE_std = []

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_5_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_10_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_20_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_30_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_40_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = mean_median(pulse_5_fold_bv_50_perc, 'peripheral_pulse_rate', 'median')
BV_5_fold_median_MAE_mean.append(mae_m)
BV_5_fold_median_MAE_std.append(mae_s)
BV_5_fold_median_MSE_mean.append(mse_m)
BV_5_fold_median_MSE_std.append(mse_s)

pulse_5_fold_bv_median = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_median_MAE_mean,
                   'MAE_std':BV_5_fold_median_MAE_std,
                   'MSE_mean': BV_5_fold_median_MSE_mean,
                   'MSE_std':BV_5_fold_median_MSE_std}
                  )
pulse_5_fold_bv_median.style.set_caption("Pulse - Bootstrap Validation 5 fold - MEDIAN")

In [ ]:
BV_Lin_MAE_mean = []
BV_Lin_MAE_std = []
BV_Lin_MSE_mean = []
BV_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_5_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_10_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_20_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_30_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_40_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_50_perc, linear_model, 'peripheral_pulse_rate')
BV_Lin_MAE_mean.append(mae_m)
BV_Lin_MAE_std.append(mae_s)
BV_Lin_MSE_mean.append(mse_m)
BV_Lin_MSE_std.append(mse_s)

pulse_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_Lin_MAE_mean,
                   'MAE_std':BV_Lin_MAE_std,
                   'MSE_mean': BV_Lin_MSE_mean,
                   'MSE_std':BV_Lin_MSE_std}
                  )
pulse_bv_linear.style.set_caption("pulse - Bootstrap Validation - LINEAR REGRESSION")


In [ ]:
BV_O_Lin_MAE_mean = []
BV_O_Lin_MAE_std = []
BV_O_Lin_MSE_mean = []
BV_O_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_5_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_10_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_20_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_30_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_40_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_50_perc, linear_model, 'peripheral_pulse_rate')
BV_O_Lin_MAE_mean.append(mae_m)
BV_O_Lin_MAE_std.append(mae_s)
BV_O_Lin_MSE_mean.append(mse_m)
BV_O_Lin_MSE_std.append(mse_s)

pulse_bv_o_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_Lin_MAE_mean,
                   'MAE_std':BV_O_Lin_MAE_std,
                   'MSE_mean': BV_O_Lin_MSE_mean,
                   'MSE_std':BV_O_Lin_MSE_std}
                  )
pulse_bv_o_linear.style.set_caption("Pulse - Bootstrap Validation (Original) - LINEAR REGRESSION")

In [ ]:
BV_5_fold_Lin_MAE_mean = []
BV_5_fold_Lin_MAE_std = []
BV_5_fold_Lin_MSE_mean = []
BV_5_fold_Lin_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_5_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_10_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_20_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_30_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_40_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_50_perc, linear_model, 'peripheral_pulse_rate')
BV_5_fold_Lin_MAE_mean.append(mae_m)
BV_5_fold_Lin_MAE_std.append(mae_s)
BV_5_fold_Lin_MSE_mean.append(mse_m)
BV_5_fold_Lin_MSE_std.append(mse_s)

pulse_5_fold_bv_linear = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_Lin_MAE_mean,
                   'MAE_std':BV_5_fold_Lin_MAE_std,
                   'MSE_mean': BV_5_fold_Lin_MSE_mean,
                   'MSE_std':BV_5_fold_Lin_MSE_std}
                  )
pulse_5_fold_bv_linear.style.set_caption("Pulse - Bootstrap Validation 5 fold - LINEAR REGRESSION")

In [ ]:
BV_RF_MAE_mean = []
BV_RF_MAE_std = []
BV_RF_MSE_mean = []
BV_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_5_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_10_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_20_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_30_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_40_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_50_perc, random_forest_model, 'peripheral_pulse_rate')
BV_RF_MAE_mean.append(mae_m)
BV_RF_MAE_std.append(mae_s)
BV_RF_MSE_mean.append(mse_m)
BV_RF_MSE_std.append(mse_s)

pulse_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_RF_MAE_mean,
                   'MAE_std':BV_RF_MAE_std,
                   'MSE_mean': BV_RF_MSE_mean,
                   'MSE_std':BV_RF_MSE_std}
                  )
pulse_bv_random_forest.style.set_caption("pulse - Bootstrap Validation - RANDOM FOREST REGRESSION")


In [ ]:
BV_O_RF_MAE_mean = []
BV_O_RF_MAE_std = []
BV_O_RF_MSE_mean = []
BV_O_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_5_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_10_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_20_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_30_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_40_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_50_perc, random_forest_model, 'peripheral_pulse_rate')
BV_O_RF_MAE_mean.append(mae_m)
BV_O_RF_MAE_std.append(mae_s)
BV_O_RF_MSE_mean.append(mse_m)
BV_O_RF_MSE_std.append(mse_s)

pulse_bv_o_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_RF_MAE_mean,
                   'MAE_std':BV_O_RF_MAE_std,
                   'MSE_mean': BV_O_RF_MSE_mean,
                   'MSE_std':BV_O_RF_MSE_std}
                  )
pulse_bv_o_random_forest.style.set_caption("Pulse - Bootstrap Validation (Original) - RANDOM FOREST REGRESSION")

In [ ]:
BV_5_fold_RF_MAE_mean = []
BV_5_fold_RF_MAE_std = []
BV_5_fold_RF_MSE_mean = []
BV_5_fold_RF_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_5_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_10_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_20_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_30_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_40_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_50_perc, random_forest_model, 'peripheral_pulse_rate')
BV_5_fold_RF_MAE_mean.append(mae_m)
BV_5_fold_RF_MAE_std.append(mae_s)
BV_5_fold_RF_MSE_mean.append(mse_m)
BV_5_fold_RF_MSE_std.append(mse_s)

pulse_5_fold_bv_random_forest = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_RF_MAE_mean,
                   'MAE_std':BV_5_fold_RF_MAE_std,
                   'MSE_mean': BV_5_fold_RF_MSE_mean,
                   'MSE_std':BV_5_fold_RF_MSE_std}
                  )
pulse_5_fold_bv_random_forest.style.set_caption("Pulse - Bootstrap Validation 5 fold - RANDOM FOREST REGRESSION")

In [ ]:
BV_XGB_MAE_mean = []
BV_XGB_MAE_std = []
BV_XGB_MSE_mean = []
BV_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_5_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_10_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_20_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_30_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_40_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_50_perc, xgboost_model, 'peripheral_pulse_rate')
BV_XGB_MAE_mean.append(mae_m)
BV_XGB_MAE_std.append(mae_s)
BV_XGB_MSE_mean.append(mse_m)
BV_XGB_MSE_std.append(mse_s)

pulse_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_XGB_MAE_mean,
                   'MAE_std':BV_XGB_MAE_std,
                   'MSE_mean': BV_XGB_MSE_mean,
                   'MSE_std':BV_XGB_MSE_std}
                  )
pulse_bv_XGB.style.set_caption("pulse - Bootstrap Validation - BOOSTING")


In [ ]:
BV_O_XGB_MAE_mean = []
BV_O_XGB_MAE_std = []
BV_O_XGB_MSE_mean = []
BV_O_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_5_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_10_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_20_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_30_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_40_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_50_perc, xgboost_model, 'peripheral_pulse_rate')
BV_O_XGB_MAE_mean.append(mae_m)
BV_O_XGB_MAE_std.append(mae_s)
BV_O_XGB_MSE_mean.append(mse_m)
BV_O_XGB_MSE_std.append(mse_s)

pulse_bv_o_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_XGB_MAE_mean,
                   'MAE_std':BV_O_XGB_MAE_std,
                   'MSE_mean': BV_O_XGB_MSE_mean,
                   'MSE_std':BV_O_XGB_MSE_std}
                  )
pulse_bv_o_XGB.style.set_caption("Pulse - Bootstrap Validation (Original) - BOOSTING")

In [ ]:
BV_5_fold_XGB_MAE_mean = []
BV_5_fold_XGB_MAE_std = []
BV_5_fold_XGB_MSE_mean = []
BV_5_fold_XGB_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_5_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_10_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_20_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_30_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_40_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_50_perc, xgboost_model, 'peripheral_pulse_rate')
BV_5_fold_XGB_MAE_mean.append(mae_m)
BV_5_fold_XGB_MAE_std.append(mae_s)
BV_5_fold_XGB_MSE_mean.append(mse_m)
BV_5_fold_XGB_MSE_std.append(mse_s)

pulse_5_fold_bv_XGB = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_XGB_MAE_mean,
                   'MAE_std':BV_5_fold_XGB_MAE_std,
                   'MSE_mean': BV_5_fold_XGB_MSE_mean,
                   'MSE_std':BV_5_fold_XGB_MSE_std}
                  )
pulse_5_fold_bv_XGB.style.set_caption("Pulse - Bootstrap Validation 5 fold - BOOSTING")

In [ ]:
BV_BAG_MAE_mean = []
BV_BAG_MAE_std = []
BV_BAG_MSE_mean = []
BV_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_5_perc, bagging_model, 'peripheral_pulse_rate')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_10_perc, bagging_model, 'peripheral_pulse_rate')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_20_perc, bagging_model, 'peripheral_pulse_rate') 
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_30_perc, bagging_model, 'peripheral_pulse_rate')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_40_perc, bagging_model, 'peripheral_pulse_rate')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_50_perc, bagging_model, 'peripheral_pulse_rate')
BV_BAG_MAE_mean.append(mae_m)
BV_BAG_MAE_std.append(mae_s)
BV_BAG_MSE_mean.append(mse_m)
BV_BAG_MSE_std.append(mse_s)

pulse_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_BAG_MAE_mean,
                   'MAE_std':BV_BAG_MAE_std,
                   'MSE_mean': BV_BAG_MSE_mean,
                   'MSE_std':BV_BAG_MSE_std}
                  )
pulse_bv_BAG.style.set_caption("pulse - Bootstrap Validation - BAGGING")


In [ ]:
BV_O_BAG_MAE_mean = []
BV_O_BAG_MAE_std = []
BV_O_BAG_MSE_mean = []
BV_O_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_5_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_10_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_20_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_30_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_40_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_bv_o_50_perc, bagging_model, 'peripheral_pulse_rate')
BV_O_BAG_MAE_mean.append(mae_m)
BV_O_BAG_MAE_std.append(mae_s)
BV_O_BAG_MSE_mean.append(mse_m)
BV_O_BAG_MSE_std.append(mse_s)

pulse_bv_o_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_O_BAG_MAE_mean,
                   'MAE_std':BV_O_BAG_MAE_std,
                   'MSE_mean': BV_O_BAG_MSE_mean,
                   'MSE_std':BV_O_BAG_MSE_std}
                  )
pulse_bv_o_BAG.style.set_caption("Pulse - Bootstrap Validation (Original) - BAGGING")

In [ ]:
BV_5_fold_BAG_MAE_mean = []
BV_5_fold_BAG_MAE_std = []
BV_5_fold_BAG_MSE_mean = []
BV_5_fold_BAG_MSE_std = []

mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_5_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_10_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_20_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_30_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_40_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)
mae_m, mae_s, mse_m, mse_s = train_test_wo_null(pulse_5_fold_bv_50_perc, bagging_model, 'peripheral_pulse_rate')
BV_5_fold_BAG_MAE_mean.append(mae_m)
BV_5_fold_BAG_MAE_std.append(mae_s)
BV_5_fold_BAG_MSE_mean.append(mse_m)
BV_5_fold_BAG_MSE_std.append(mse_s)

pulse_5_fold_bv_BAG = pd.DataFrame({'% Missing': [5,10,20,30,40,50],
    'MAE_mean': BV_5_fold_BAG_MAE_mean,
                   'MAE_std':BV_5_fold_BAG_MAE_std,
                   'MSE_mean': BV_5_fold_BAG_MSE_mean,
                   'MSE_std':BV_5_fold_BAG_MSE_std}
                  )
pulse_5_fold_bv_BAG.style.set_caption("Pulse - Bootstrap Validation 5 fold - BAGGING")

# Saving Values in pkl files

In [ ]:
# saving dbp values in pkl
import pickle
dbp_values = [
    dbp_5_fold_mean, 
    dbp_5_fold_median, 
    dbp_5_fold_linear,
    dbp_5_fold_random_forest,
    dbp_5_fold_XGB,
    dbp_5_fold_BAG,
    dbp_10_fold_mean,
    dbp_10_fold_median,
    dbp_10_fold_linear,
    dbp_10_fold_random_forest,
    dbp_10_fold_XGB,
    dbp_10_fold_BAG,
    dbp_bv_mean,
    dbp_bv_o_mean,
    dbp_5_fold_bv_mean,
    dbp_bv_median,
    dbp_bv_o_median,
    dbp_5_fold_bv_median,
    dbp_bv_linear,
    dbp_bv_o_linear,
    dbp_5_fold_bv_linear,
    dbp_bv_random_forest,
    dbp_bv_o_random_forest,
    dbp_5_fold_bv_random_forest,
    dbp_bv_XGB,
    dbp_bv_o_XGB,
    dbp_5_fold_bv_XGB,
    dbp_bv_BAG,
    dbp_bv_o_BAG,
    dbp_5_fold_bv_BAG]
with open('dbp_values.pkl', 'wb') as f:
    pickle.dump(dbp_values, f)

# to access
# with open('dbp_values.pkl', 'rb') as f:
#     dbp_values = pickle.load(f)

In [ ]:
# saving sbp values in pkl
sbp_values = [
    sbp_5_fold_mean, 
    sbp_5_fold_median, 
    sbp_5_fold_linear,
    sbp_5_fold_random_forest,
    sbp_5_fold_XGB,
    sbp_5_fold_BAG,
    sbp_10_fold_mean,
    sbp_10_fold_median,
    sbp_10_fold_linear,
    sbp_10_fold_random_forest,
    sbp_10_fold_XGB,
    sbp_10_fold_BAG,
    sbp_bv_mean,
    sbp_bv_o_mean,
    sbp_5_fold_bv_mean,
    sbp_bv_median,
    sbp_bv_o_median,
    sbp_5_fold_bv_median,
    sbp_bv_linear,
    sbp_bv_o_linear,
    sbp_5_fold_bv_linear,
    sbp_bv_random_forest,
    sbp_bv_o_random_forest,
    sbp_5_fold_bv_random_forest,
    sbp_bv_XGB,
    sbp_bv_o_XGB,
    sbp_5_fold_bv_XGB,
    sbp_bv_BAG,
    sbp_bv_o_BAG,
    sbp_5_fold_bv_BAG]
with open('sbp_values.pkl', 'wb') as f:
    pickle.dump(sbp_values, f)

# to access
# with open('sbp_values.pkl', 'rb') as f:
#     sbp_values = pickle.load(f)

In [ ]:
# saving resp values in pkl
resp_values = [
    resp_5_fold_mean, 
    resp_5_fold_median, 
    resp_5_fold_linear,
    resp_5_fold_random_forest,
    resp_5_fold_XGB,
    resp_5_fold_BAG,
    resp_10_fold_mean,
    resp_10_fold_median,
    resp_10_fold_linear,
    resp_10_fold_random_forest,
    resp_10_fold_XGB,
    resp_10_fold_BAG,
    resp_bv_mean,
    resp_bv_o_mean,
    resp_5_fold_bv_mean,
    resp_bv_median,
    resp_bv_o_median,
    resp_5_fold_bv_median,
    resp_bv_linear,
    resp_bv_o_linear,
    resp_5_fold_bv_linear,
    resp_bv_random_forest,
    resp_bv_o_random_forest,
    resp_5_fold_bv_random_forest,
    resp_bv_XGB,
    resp_bv_o_XGB,
    resp_5_fold_bv_XGB,
    resp_bv_BAG,
    resp_bv_o_BAG,
    resp_5_fold_bv_BAG]
with open('resp_values.pkl', 'wb') as f:
    pickle.dump(resp_values, f)

# to access
# with open('resp_values.pkl', 'rb') as f:
#     resp_values = pickle.load(f)

In [ ]:
# saving pulse values in pkl
pulse_values = [
    pulse_5_fold_mean, 
    pulse_5_fold_median, 
    pulse_5_fold_linear,
    pulse_5_fold_random_forest,
    pulse_5_fold_XGB,
    pulse_5_fold_BAG,
    pulse_10_fold_mean,
    pulse_10_fold_median,
    pulse_10_fold_linear,
    pulse_10_fold_random_forest,
    pulse_10_fold_XGB,
    pulse_10_fold_BAG,
    pulse_bv_mean,
    pulse_bv_o_mean,
    pulse_5_fold_bv_mean,
    pulse_bv_median,
    pulse_bv_o_median,
    pulse_5_fold_bv_median,
    pulse_bv_linear,
    pulse_bv_o_linear,
    pulse_5_fold_bv_linear,
    pulse_bv_random_forest,
    pulse_bv_o_random_forest,
    pulse_5_fold_bv_random_forest,
    pulse_bv_XGB,
    pulse_bv_o_XGB,
    pulse_5_fold_bv_XGB,
    pulse_bv_BAG,
    pulse_bv_o_BAG,
    pulse_5_fold_bv_BAG]
# with open('pulse_values.pkl', 'wb') as f:
#     pickle.dump(pulse_values, f)

# to access
with open('pulse_values.pkl', 'rb') as f:
    pulse_values = pickle.load(f)

# plots

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_5_fold_random_forest['MAE_mean']
s2 = dbp_5_fold_XGB['MAE_mean']
s3 = dbp_5_fold_BAG['MAE_mean']
s4 = dbp_5_fold_mean['MAE_mean']
s5 = dbp_5_fold_median['MAE_mean']
s6 = dbp_5_fold_linear['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag", "Mean", "Median", "Linear"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='About as simple as it gets, folks')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_5_fold_random_forest['MAE_mean']
s2 = dbp_5_fold_XGB['MAE_mean']
s3 = dbp_5_fold_BAG['MAE_mean']
s6 = dbp_5_fold_linear['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag", "Linear"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='About as simple as it gets, folks')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_10_fold_random_forest['MAE_mean']
s2 = dbp_10_fold_XGB['MAE_mean']
s3 = dbp_10_fold_BAG['MAE_mean']
# s4 = five_fold_RF_MAE_mean
# s5 = five_fold_XGB_MAE_mean
# s6 = five_fold_BAG_MAE_mean
fig, ax = plt.subplots()
classes = ['Random Forest',"Boosting", "Bagging"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
# ax.plot(t, s4)
# ax.plot(t, s5)
# ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='10-Fold Cross Validation - DBP')
ax.grid()

fig.savefig("test.png")
plt.legend(title="Best 3 methods", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_5_fold_BAG['MAE_mean']
s2 = dbp_10_fold_BAG['MAE_mean']
s3 = dbp_bv_o_BAG['MAE_mean']
s4 = dbp_bv_BAG['MAE_mean']
s5 = dbp_5_fold_bv_BAG['MAE_mean']
fig, ax = plt.subplots()
classes = ['5 fold',"10 fold", "bv", "bv ft", "bv kf"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.plot(t, s4)
ax.plot(t, s5)

ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='About as simple as it gets, folks')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_bv_random_forest['MAE_mean']
s2 = dbp_bv_XGB['MAE_mean']
s3 = dbp_bv_BAG['MAE_mean']
s4 = dbp_bv_linear['MAE_mean']
s5 = dbp_bv_median['MAE_mean']
s6 = dbp_bv_mean['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag", "Linear", "median", "mean"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='Bootstrap Validation - DBP')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = dbp_bv_random_forest['MAE_mean']
s2 = dbp_bv_XGB['MAE_mean']
s3 = dbp_bv_BAG_temp['MAE_mean']
# s4 = dbp_bv_linear['MAE_mean']
# s5 = dbp_bv_median['MAE_mean']
# s6 = dbp_bv_mean['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag"]#, "Linear", "median", "mean"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
# ax.plot(t, s4)
# ax.plot(t, s5)
# ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='Bootstrap Validation - DBP (3 best)')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = sbp_10_fold_random_forest['MAE_mean']
s2 = sbp_10_fold_XGB['MAE_mean']
s3 = sbp_10_fold_BAG['MAE_mean']
# s4 = dbp_bv_linear['MAE_mean']
# s5 = dbp_bv_median['MAE_mean']
# s6 = dbp_bv_mean['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag"]#, "Linear", "median", "mean"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
# ax.plot(t, s4)
# ax.plot(t, s5)
# ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='10 fold cv - SBP (3 best)')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = sbp_bv_random_forest['MAE_mean']
s2 = sbp_bv_XGB_temp['MAE_mean']
s3 = sbp_bv_BAG['MAE_mean']
# s4 = dbp_bv_linear['MAE_mean']
# s5 = dbp_bv_median['MAE_mean']
# s6 = dbp_bv_mean['MAE_mean']
fig, ax = plt.subplots()
classes = ['RF',"boost", "bag"]#, "Linear", "median", "mean"]

ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
# ax.plot(t, s4)
# ax.plot(t, s5)
# ax.plot(t, s6)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='Bootstrap Validation - SBP (3 best)')
ax.grid()

fig.savefig("test.png")
plt.legend(title="test", labels=classes)
plt.show()

In [ ]:
# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = resp_10_fold_mean['MAE_mean']
s2 = resp_10_fold_median['MAE_mean']
s3 = resp_10_fold_linear['MAE_mean']
s4 = resp_10_fold_random_forest['MAE_mean']
s5 = resp_10_fold_XGB['MAE_mean']
s6 = resp_10_fold_BAG['MAE_mean']
# s4 = five_fold_RF_MAE_mean
# s5 = five_fold_XGB_MAE_mean
# s6 = five_fold_BAG_MAE_mean
fig, ax = plt.subplots()
classes = ['Random Forest',"Boosting", "Bagging", 'mean','median','Linear Regression']


ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='10-Fold Cross Validation - Respiratory Rate ')
ax.grid()

fig.savefig("test.png")
plt.legend(title="Methods (3 best)", labels=classes)
plt.show()

In [ ]:
# Data for plotting
t = np.array([5,10,20,30,40,50])
# s1 = resp_10_fold_mean['MAE_mean']
# s2 = resp_10_fold_median['MAE_mean']
# s3 = resp_10_fold_linear['MAE_mean']
s4 = resp_bv_random_forest['MAE_mean']
s5 = resp_bv_XGB['MAE_mean']
s6 = resp_bv_BAG['MAE_mean']
# s4 = five_fold_RF_MAE_mean
# s5 = five_fold_XGB_MAE_mean
# s6 = five_fold_BAG_MAE_mean
fig, ax = plt.subplots()
classes = ['Random Forest',"Boosting", "Bagging"] #, 'mean','median','Linear Regression']


ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
# ax.plot(t,s1)
# ax.plot(t, s2)
# ax.plot(t, s3)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='Bootstrap Validation - Respiratory Rate ')
ax.grid()

fig.savefig("test.png")
plt.legend(title="Methods (3 best)", labels=classes)
plt.show()

In [ ]:
# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = resp_10_fold_mean['MAE_mean']
s2 = resp_10_fold_median['MAE_mean']
s3 = resp_10_fold_linear['MAE_mean']
s4 = resp_10_fold_random_forest['MAE_mean']
s5 = resp_10_fold_XGB['MAE_mean']
s6 = resp_10_fold_BAG['MAE_mean']

fig, ax = plt.subplots()
classes = ['Random Forest',"Boosting", "Bagging", 'mean','median','Linear Regression']


ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='10-Fold Cross Validation - Respiratory Rate ')
ax.grid()

fig.savefig("test.png")
plt.legend(title="Methods", labels=classes)
plt.show()

In [ ]:
# Data for plotting
t = np.array([5,10,20,30,40,50])
s1 = resp_bv_mean['MAE_mean']
s2 = resp_bv_median['MAE_mean']
s3 = resp_bv_linear['MAE_mean']
s4 = resp_bv_random_forest['MAE_mean']
s5 = resp_bv_XGB['MAE_mean']
s6 = resp_bv_BAG['MAE_mean']
# s4 = five_fold_RF_MAE_mean
# s5 = five_fold_XGB_MAE_mean
# s6 = five_fold_BAG_MAE_mean
fig, ax = plt.subplots()
classes = ['Random Forest',"Boosting", "Bagging", 'mean','median','Linear Regression']


ax.plot(t, s4)
ax.plot(t, s5)
ax.plot(t, s6)
ax.plot(t,s1)
ax.plot(t, s2)
ax.plot(t, s3)
ax.set(xlabel='Percentage missing', ylabel='Mean Absolute Error',
       title='Bootstrap Validation - Respiratory Rate ')
ax.grid()

fig.savefig("test.png")
plt.legend(title="Methods", labels=classes)
plt.show()

In [ ]:
sbp_5_fold_XGB['MAE_mean'].tolist() + sbp_10_fold_XGB['MAE_mean'].tolist() + sbp_bv_XGB['MAE_mean'].tolist()
sum(sbp)/len(sbp)

In [ ]:
dbp = dbp_5_fold_XGB['MAE_mean'].tolist() + dbp_10_fold_XGB['MAE_mean'].tolist() + dbp_bv_XGB['MAE_mean'].tolist()
dbp_b = sum(dbp)/len(dbp)    
sbp = sbp_5_fold_XGB['MAE_mean'].tolist() + sbp_10_fold_XGB['MAE_mean'].tolist() + sbp_bv_XGB['MAE_mean'].tolist()
sbp_b = sum(sbp)/len(sbp)
resp = resp_5_fold_XGB['MAE_mean'].tolist() + resp_10_fold_XGB['MAE_mean'].tolist() + resp_bv_XGB['MAE_mean'].tolist()
resp_b = sum(resp)/len(resp)
pulse = pulse_5_fold_XGB['MAE_mean'].tolist() + pulse_10_fold_XGB['MAE_mean'].tolist() + pulse_bv_XGB['MAE_mean'].tolist()
pulse_b = sum(pulse)/len(pulse)

dbp = dbp_5_fold_mean['MAE_mean'].tolist() + dbp_10_fold_mean['MAE_mean'].tolist() + dbp_bv_mean['MAE_mean'].tolist()
dbp_m = sum(dbp)/len(dbp)    
sbp = sbp_5_fold_mean['MAE_mean'].tolist() + sbp_10_fold_mean['MAE_mean'].tolist() + sbp_bv_mean['MAE_mean'].tolist()
sbp_m = sum(sbp)/len(sbp)
resp = resp_5_fold_mean['MAE_mean'].tolist() + resp_10_fold_mean['MAE_mean'].tolist() + resp_bv_mean['MAE_mean'].tolist()
resp_m = sum(resp)/len(resp)
pulse = pulse_5_fold_mean['MAE_mean'].tolist() + pulse_10_fold_mean['MAE_mean'].tolist() + pulse_bv_mean['MAE_mean'].tolist()
pulse_m = sum(pulse)/len(pulse)

In [ ]:
dbp_b

In [ ]:
fig, ax = plt.subplots()

labels = ['DBP', 'SBP', 'Respiratory Rate', 'Pulse']
counts = [dbp_b, sbp_b, resp_b, pulse_b]

ax.bar(labels, counts)#, label=bar_labels, color=bar_colors)

ax.set_ylabel('Mean Absolute Error')
ax.set_title('Avegrage MAE values in Boosting')
# ax.legend(title='Fruit color')

plt.show()

In [ ]:

fig, ax = plt.subplots()

labels = ['DBP', 'SBP', 'Respiratory Rate', 'Pulse']
dbp = dbp_5_fold_XGB['MSE_mean'].tolist() + dbp_10_fold_XGB['MSE_mean'].tolist() + dbp_bv_XGB['MSE_mean'].tolist()
dbp_s = sum(dbp)/len(dbp)    
sbp = sbp_5_fold_XGB['MSE_mean'].tolist() + sbp_10_fold_XGB['MSE_mean'].tolist() + sbp_bv_XGB['MSE_mean'].tolist()
sbp_s = sum(sbp)/len(sbp)
resp = resp_5_fold_XGB['MSE_mean'].tolist() + resp_10_fold_XGB['MSE_mean'].tolist() + resp_bv_XGB['MSE_mean'].tolist()
resp_s = sum(resp)/len(resp)
pulse = pulse_5_fold_XGB['MSE_mean'].tolist() + pulse_10_fold_XGB['MSE_mean'].tolist() + pulse_bv_XGB['MSE_mean'].tolist()
pulse_s = sum(pulse)/len(pulse)
counts = [dbp_s, sbp_s, resp_s, pulse_s]

ax.bar(labels, counts)#, label=bar_labels, color=bar_colors)

ax.set_ylabel('Mean Squared Error')
ax.set_title('Avegrage MSE values in Boosting')
# ax.legend(title='Fruit color')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def hat_graph(ax, xlabels, values, group_labels):
    """
    Create a hat graph.

    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The Axes to plot into.
    xlabels : list of str
        The category names to be displayed on the x-axis.
    values : (M, N) array-like
        The data values.
        Rows are the groups (len(group_labels) == M).
        Columns are the categories (len(xlabels) == N).
    group_labels : list of str
        The group labels displayed in the legend.
    """

    def label_bars(heights, rects):
        """Attach a text label on top of each bar."""
        for height, rect in zip(heights, rects):
            ax.annotate(f'{height}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 4),  # 4 points vertical offset.
                        textcoords='offset points',
                        ha='center', va='bottom')

    values = np.asarray(values)
    x = np.arange(values.shape[1])
    ax.set_xticks(x, labels=xlabels)
    spacing = 0.3  # spacing between hat groups
    width = (1 - spacing) / values.shape[0]
    heights0 = values[0]
    for i, (heights, group_label) in enumerate(zip(values, group_labels)):
        style = {'fill': False} if i == 0 else {'edgecolor': 'black'}
        rects = ax.bar(x - spacing/2 + i * width, heights - heights0,
                       width, bottom=heights0, label=group_label, **style)
        label_bars(heights, rects)


# initialise labels and a numpy array make sure you have
# N labels of N number of values in the array
xlabels = ['DBP', 'SBP', 'Respiratory Rate', 'Pulse']
playerA = np.array([round(dbp_b, 2), round(sbp_b, 2), round(resp_b, 2), round(pulse_b, 2)])
playerB = np.array([round(dbp_m, 2), round(sbp_m, 2), round(resp_m, 2), round(pulse_m, 2)])

fig, ax = plt.subplots()
hat_graph(ax, xlabels, [playerA, playerB], ['Player A', 'Player B'])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('')
ax.set_ylabel('Mean Absolute Error')
ax.set_ylim(0, 30)
ax.set_title('Range of MAE from XGBoost to Mean imputation method method')
# ax.legend()

fig.tight_layout()
plt.show()

Step 1: A simple imputation, such as imputing the mean, is performed for every missing value in the dataset. These mean imputations can be thought of as “place holders.”

Step 2: The “place holder” mean imputations for one variable (“var”) are set back to missing.

Step 3: The observed values from the variable “var” in Step 2 are regressed on the other variables in the imputation model, which may or may not consist of all of the variables in the dataset. In other words, “var” is the dependent variable in a regression model and all the other variables are independent variables in the regression model. These regression models operate under the same assumptions that one would make when performing linear, logistic, or Poison regression models outside of the context of imputing missing data.

Step 4: The missing values for “var” are then replaced with predictions (imputations) from the regression model. When “var” is subsequently used as an independent variable in the regression models for other variables, both the observed and these imputed values will be used.

Step 5: Steps 2–4 are then repeated for each variable that has missing data. The cycling through each of the variables constitutes one iteration or “cycle.” At the end of one cycle all of the missing values have been replaced with predictions from regressions that reflect the relationships observed in the data.

Step 6: Steps 2–4 are repeated for a number of cycles, with the imputations being updated at each cycle.

#Explanation with Example

Step 1: Replace (or impute) the missing values in each variable with temporary "place holder" values derived solely from the non-missing values available for that variable. For example, replace the missing age value with the mean age value observed in the data, replace the missing income values with the mean income value observed in the data, etc.

Step 2 Set back to missing the “place holder” imputations for the age variable only. This way, the current data copy contains missing values for age, but not for income and gender.

Step 3: Regress age on income and gender via a linear regression model (though it is possible to also regress age on only one of these variables); to be able to fit the model to the current data copy, drop all the records where age is missing during the model fitting process. In this model, age is the dependent variable and income and gender are the independent variables.

Step 4 Use the fitted regression model in the previous step to predict the missing age values. (When age will be subsequently used as an independent variable in the regression models for other variables, both the observed values of age and these predicted values will be used.) The article doesn't make it clear that a random component should be added to these predictions.

Step 5: Repeat Steps 2–4 separately for each variable that has missing data, namely income and gender.

